In [ ]:
!pip install requests pandas protobuf

In [ ]:
import requests

API_KEY = "EjtGzn2BsTmH0ihW7gcm8mA2g4OfSKlC"
API_URL = f"https://otd.delhi.gov.in/api/realtime/VehiclePositions.pb?key=EjtGzn2BsTmH0ihW7gcm8mA2g4OfSKlC"

# Fetch the data
response = requests.get("https://otd.delhi.gov.in/api/realtime/VehiclePositions.pb?key=EjtGzn2BsTmH0ihW7gcm8mA2g4OfSKlC")

with open("VehiclePositions.pb", "wb") as f:
    f.write(response.content)

print("Downloaded VehiclePositions.pb successfully!")


Downloaded VehiclePositions.pb successfully!


In [ ]:
!pip install gtfs-realtime-bindings

  Preparing metadata (setup.py) ... done
  Created wheel for gtfs-realtime-bindings: filename=gtfs_realtime_bindings-1.0.0-py3-none-any.whl size=5987 sha256=d0cfecb7d7964c44a71d3208b42138be0c6545adc25242f1f7efcf0700a0f888
  Stored in directory: /root/.cache/pip/wheels/de/77/98/c97c7fb5e3a7359e3f83c9c4a62115d3700e9b5445854ca122
Successfully built gtfs-realtime-bindings


In [ ]:
from google.transit import gtfs_realtime_pb2
import pandas as pd

# Read the downloaded .pb file
feed = gtfs_realtime_pb2.FeedMessage()
with open("VehiclePositions.pb", "rb") as f:
    feed.ParseFromString(f.read())

# Extract bus positions
bus_data = []
for entity in feed.entity:
    if entity.HasField("vehicle"):
        vehicle = entity.vehicle
        bus_data.append({
            "Vehicle ID": vehicle.vehicle.id,
            "Trip ID": vehicle.trip.trip_id,
            "Latitude": vehicle.position.latitude,
            "Longitude": vehicle.position.longitude,
            "Timestamp": vehicle.timestamp
        })

# Convert to DataFrame for better visualization
df = pd.DataFrame(bus_data)
print(df.head(100))  # Show the first few rows


   Vehicle ID      Trip ID   Latitude  Longitude   Timestamp
0   DL1PD0299  10958_23_15  28.644241  77.256294  1745776641
1   DL1PD6177    674_23_37  28.537613  76.914345  1745776657
2   DL1PD0255  10907_23_16  28.620272  76.930183  1745776777
3   DL1PD5378   1796_23_14  28.623579  77.102051  1745776847
4   DL1PD4904    381_21_46  28.856642  77.052773  1745776858
..        ...          ...        ...        ...         ...
95  DL1PD5778  10017_22_57  28.538551  76.914223  1745777385
96  DL1PD5053   1174_22_11  28.708668  77.027847  1745777385
97  DL1PD4712    408_23_14  28.623013  77.102226  1745777385
98  DL1PD5282   1796_22_42  28.624884  77.100708  1745777385
99  DL1PD5982   1720_21_54  28.538992  76.913841  1745777385

[100 rows x 5 columns]


In [ ]:
import requests
import json

# Hardcoded TomTom API Key (for non-commercial use)
TOMTOM_API_KEY = "TBo0EKP7kmbDlw9UG3j8QJQTGi3nvFTk"

def get_traffic_flow(lat, lon):
    """
    Fetches real-time traffic flow data from TomTom API for a given location.

    Parameters:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.

    Returns:
        dict: JSON response with traffic flow details.
    """
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/relative/10/json?key={TOMTOM_API_KEY}&point={lat},{lon}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

def get_traffic_incidents(bbox, language="en-GB", timeValidityFilter="present", categoryFilter="1,6,8"):
    """
    Fetches real-time traffic incident data from TomTom API for a given bounding box.

    Parameters:
        bbox (str): Bounding box in the format "minLon,minLat,maxLon,maxLat".
        language (str): Output language (default "en-GB").
        timeValidityFilter (str): Filter for incident time validity (default "present").
        categoryFilter (str): Comma-separated list of incident category values. (1: Accident, 6: Jam, 8: RoadClosed)

    Returns:
        dict: JSON response containing traffic incident details.
    """
    url = (f"https://api.tomtom.com/traffic/services/5/incidentDetails?"
           f"key={TOMTOM_API_KEY}&bbox={bbox}"
           f"&fields={{incidents{{type,geometry{{type,coordinates}},properties{{iconCategory}}}}}}"
           f"&language={language}&timeValidityFilter={timeValidityFilter}&categoryFilter={categoryFilter}")

    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

# Example usage:
# Traffic Flow Data for Connaught Place, Delhi
lat, lon = 28.6139, 77.2090
traffic_flow_data = get_traffic_flow(lat, lon)

# Define a bounding box for the Delhi region ("minLon,minLat,maxLon,maxLat")
bbox = "77.10,28.60,77.30,28.70"
traffic_incidents_data = get_traffic_incidents(bbox)

# Combine and print results
result = {
    "Traffic_Flow": traffic_flow_data,
    "Traffic_Incidents": traffic_incidents_data
}

print(json.dumps(result, indent=2))


{
  "Traffic_Flow": {
    "flowSegmentData": {
      "frc": "FRC4",
      "currentSpeed": 23,
      "freeFlowSpeed": 24,
      "currentTravelTime": 280,
      "freeFlowTravelTime": 268,
      "confidence": 0.724656,
      "roadClosure": true,
      "coordinates": {
        "coordinate": [
          {
            "latitude": 28.60941906113328,
            "longitude": 77.20872623315284
          },
          {
            "latitude": 28.60976770669141,
            "longitude": 77.20874098530243
          },
          {
            "latitude": 28.6100520381346,
            "longitude": 77.20875573745201
          },
          {
            "latitude": 28.610245197643412,
            "longitude": 77.20876646628807
          },
          {
            "latitude": 28.611056537285588,
            "longitude": 77.20882011046837
          },
          {
            "latitude": 28.61121614072751,
            "longitude": 77.20882815709541
          },
          {
            "latitude": 28.6115

In [ ]:
!pip install openmeteo-requests requests-cache retry-requests numpy pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 4.3 MB/s eta 0:00:00


In [ ]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# API URL for historical weather data
url = "https://archive-api.open-meteo.com/v1/archive"

# Delhi coordinates and selected weather variables
params = {
    "latitude": 28.6139,    # Delhi's latitude
    "longitude": 77.2090,   # Delhi's longitude
    "start_date": "2010-01-01",
    "end_date": "2024-01-31",
    "hourly": ["temperature_2m", "precipitation", "wind_speed_10m", "cloudcover"]
}

# Fetch data from the Open-Meteo API
responses = openmeteo.weather_api(url, params=params)
response = responses[0]

# Note: The API returns data in UTC, and timezone info might not be provided.
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print("Timezone: Data not provided by API, assuming UTC")
print("UTC Offset: 0 s (Data in UTC)")

# Process hourly data
hourly = response.Hourly()
utc_date_range = pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left"
)

# Manually convert from UTC to IST (UTC+5:30)
ist_date_range = utc_date_range.tz_convert("Asia/Kolkata")

hourly_data = {
    "date": ist_date_range,
    "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
    "precipitation": hourly.Variables(1).ValuesAsNumpy(),
    "wind_speed_10m": hourly.Variables(2).ValuesAsNumpy(),
    "cloudcover": hourly.Variables(3).ValuesAsNumpy(),
}

# Convert to DataFrame and save only the weather data as CSV
hourly_dataframe = pd.DataFrame(data=hourly_data)
hourly_dataframe.to_csv("historical_weather_jan2024.csv", index=False)

print("Historical weather data saved as 'historical_weather_jan2024.csv'")
print(hourly_dataframe.head())


Coordinates: 28.646747589111328°N 77.17218017578125°E
Elevation: 214.0 m asl
Timezone: Data not provided by API, assuming UTC
UTC Offset: 0 s (Data in UTC)
Historical weather data saved as 'historical_weather_jan2024.csv'
                       date  temperature_2m  precipitation  wind_speed_10m  \
0 2010-01-01 05:30:00+05:30        9.197500            0.0       10.464797   
1 2010-01-01 06:30:00+05:30        8.947500            0.0        9.511088   
2 2010-01-01 07:30:00+05:30        8.747499            0.0        8.854829   
3 2010-01-01 08:30:00+05:30       10.697500            0.0       10.041354   
4 2010-01-01 09:30:00+05:30       14.597500            0.0        8.759178   

   cloudcover  
0         0.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0  


In [ ]:
# Weather Forecast Data

import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup API client with cache and retry mechanisms
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# API URL and parameters
url = "https://api.open-meteo.com/v1/forecast"
params = {
    "latitude": 28.6139,  # Example: New Delhi, India
    "longitude": 77.2090,
    "current": "temperature_2m,wind_speed_10m",
    "hourly": "temperature_2m,precipitation",
    "forecast_days": 1,
    "timezone": "auto"
}

# Fetch weather data
responses = openmeteo.weather_api(url, params=params)

# Extract response data
response = responses[0]
print(f"Location: {response.Latitude()}°N, {response.Longitude()}°E")
print(f"Timezone: {response.Timezone()} {response.TimezoneAbbreviation()}")

# Current weather data
current_weather = response.Current()
print("\nCurrent Weather:")
print(f"Temperature: {current_weather.Variables(0).Value()}°C")
print(f"Wind Speed: {current_weather.Variables(1).Value()} km/h")

# Hourly forecast data
hourly = response.Hourly()
hourly_temperature = hourly.Variables(0).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(1).ValuesAsNumpy()

# Convert to DataFrame
hourly_data = {
    "date": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "temperature_2m": hourly_temperature,
    "precipitation": hourly_precipitation
}

hourly_df = pd.DataFrame(hourly_data)
print("\nHourly Forecast:")
print(hourly_df)


Location: 28.625°N, 77.25°E
Timezone: b'Asia/Kolkata' b'GMT+5:30'

Current Weather:
Temperature: 27.700000762939453°C
Wind Speed: 7.2805495262146 km/h

Hourly Forecast:
                        date  temperature_2m  precipitation
0  2025-04-26 18:30:00+00:00       27.211000            0.0
1  2025-04-26 19:30:00+00:00       26.361000            0.0
2  2025-04-26 20:30:00+00:00       25.161001            0.0
3  2025-04-26 21:30:00+00:00       24.711000            0.0
4  2025-04-26 22:30:00+00:00       24.211000            0.0
5  2025-04-26 23:30:00+00:00       25.511000            0.0
6  2025-04-27 00:30:00+00:00       24.561001            0.0
7  2025-04-27 01:30:00+00:00       25.911001            0.0
8  2025-04-27 02:30:00+00:00       28.061001            0.0
9  2025-04-27 03:30:00+00:00       30.911001            0.0
10 2025-04-27 04:30:00+00:00       33.861000            0.0
11 2025-04-27 05:30:00+00:00       36.560997            0.0
12 2025-04-27 06:30:00+00:00       38.010998       

In [ ]:
!pip install osmnx geopandas folium shapely networkx pyrosm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pyrobuf-0.9.3-cp311-cp311-linux_x86_64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.9/99.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.3 MB/s eta 0:00:00
  Created wheel for pyrosm: filename=pyrosm-0.6.2-cp311-cp311-linux_x86_64.whl size=7783023 sha256=0e2ec787c96b8b0df8d84319715616f7dde6edc955e28d842ac57e469289567a
  Stored in directory: /root/.cache/pip/wheels/f3/77/c3/7b767c229a2bba06eb2e47fbce4cdbc4490b38d97f2a0ae64d
  Created wheel for cykhash: filename=cykhash-2.0.1-cp311-cp311-linux_x86_64.whl size=3

In [ ]:
# Cell 1: Imports
# Import all necessary libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import folium
import osmnx as ox
import networkx as nx
import warnings
import time
import gc # Import garbage collector


# Suppress the specific DeprecationWarning for unary_union if desired
# warnings.filterwarnings("ignore", message="The 'unary_union' attribute is deprecated", category=DeprecationWarning)

print("Cell 1/5: Libraries imported.")

Cell 1/5: Libraries imported.


In [ ]:
# Cell 2: Configuration & File Loading
# Define target trip and load essential GTFS data with optimizations

# --- Select Trip ---
# Use the string format matching your data
TARGET_TRIP_ID = '362_13_36' # Example bus trip_id - Set the trip you want to process

print(f"--- Loading Static Files for Trip: {TARGET_TRIP_ID} ---")

# --- Define columns and dtypes for optimized loading ---
stops_cols = ['stop_id', 'stop_lat', 'stop_lon']
stops_dtypes = {'stop_id': str, 'stop_lat': 'float32', 'stop_lon': 'float32'}

stop_times_cols = ['trip_id', 'stop_id', 'stop_sequence']
# Load sequence as object first, handle conversion below robustly
stop_times_dtypes = {'trip_id': str, 'stop_id': str, 'stop_sequence': object}

# --- Load only necessary files and columns ---
try:
    stops_df = pd.read_csv('stops.txt', usecols=stops_cols, dtype=stops_dtypes)
    stop_times_df = pd.read_csv('stop_times.txt', usecols=stop_times_cols, dtype=stop_times_dtypes)
    print("Essential GTFS-like files loaded successfully.")

except FileNotFoundError as e:
    print(f"Error loading required GTFS file: {e}. Please ensure stops.txt and stop_times.txt are present.")
    # Raising an error might be better than exit() in a notebook cell
    raise e
except Exception as e:
    print(f"Error reading files: {e}")
    raise e # Re-raise the exception to stop execution clearly

print("Cell 2/5: Files loaded and configuration set.")

--- Loading Static Files for Trip: 362_13_36 ---
Essential GTFS-like files loaded successfully.
Cell 2/5: Files loaded and configuration set.


In [ ]:
# Cell: Quick Stop Visualization
# Purpose: Enter a Trip ID and visualize its stops on a map (no path routing).
# Assumes Cell 1 (Imports) and Cell 2 (File Loading) have been run successfully.

import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import folium
from IPython.display import display
import warnings
import gc # Added for cleanup consistency

# --- Configuration ---
# !!! SET THE TRIP ID YOU WANT TO VISUALIZE HERE !!!
target_trip_id_viz = TARGET_TRIP_ID # Example - Replace with a valid trip ID string from your stop_times.txt
# --------------------

print(f"--- Visualizing Stops for Trip ID: {target_trip_id_viz} ---")

# Check if necessary DataFrames exist
if 'stop_times_df' not in locals() or 'stops_df' not in locals():
    print("Error: stop_times_df or stops_df not found. Please run Cell 2 first.")
else:
    stops_gdf_viz = gpd.GeoDataFrame() # Initialize empty GeoDataFrame for this cell
    trip_stop_times_viz = None # Initialize
    stops_info_dict_viz = None # Initialize
    temp_stop_times = None # Initialize

    try:
        # Ensure stop_sequence is numeric for sorting (apply locally if needed)
        # Work on a copy to avoid modifying the original df if Cell 2 is re-run
        temp_stop_times = stop_times_df.copy()
        try:
            temp_stop_times['stop_sequence'] = pd.to_numeric(temp_stop_times['stop_sequence'], errors='coerce')
            temp_stop_times = temp_stop_times.dropna(subset=['stop_sequence'])
            temp_stop_times['stop_sequence'] = temp_stop_times['stop_sequence'].astype(int)
        except Exception as e:
             print(f"Warning: Could not process stop_sequence: {e}")

        # Filter for the target trip and sort
        trip_stop_times_viz = temp_stop_times[temp_stop_times['trip_id'] == target_trip_id_viz].sort_values('stop_sequence')

        if trip_stop_times_viz.empty:
            print(f"Error: No stop times found for trip_id '{target_trip_id_viz}'.")
        else:
            ordered_stop_ids_viz = trip_stop_times_viz['stop_id'].tolist()
            print(f"Found {len(ordered_stop_ids_viz)} stops in sequence for trip {target_trip_id_viz}.")

            # Get Stop Coordinates
            # Use a copy of stops_df as well
            temp_stops_df = stops_df.copy()
            trip_stops_info_filtered_viz = temp_stops_df[temp_stops_df['stop_id'].isin(ordered_stop_ids_viz)]
            stops_info_dict_viz = trip_stops_info_filtered_viz.set_index('stop_id').to_dict('index')

            ordered_stop_coords_viz = []
            valid_stop_ids_in_order_viz = []
            for sid in ordered_stop_ids_viz:
                stop_info = stops_info_dict_viz.get(sid)
                # Check if stop exists AND lat/lon are valid numbers
                if stop_info and isinstance(stop_info.get('stop_lat'), (int, float)) and isinstance(stop_info.get('stop_lon'), (int, float)) and pd.notna(stop_info.get('stop_lat')) and pd.notna(stop_info.get('stop_lon')):
                    ordered_stop_coords_viz.append((stop_info['stop_lat'], stop_info['stop_lon']))
                    valid_stop_ids_in_order_viz.append(sid)
                else:
                    print(f"Warning: Skipping stop_id '{sid}' due to missing/invalid coordinates.")

            print(f"Found valid coordinates for {len(valid_stop_ids_in_order_viz)} stops.")

            if len(valid_stop_ids_in_order_viz) > 0:
                 # Create GeoDataFrame for valid stops
                 stop_geometries_viz = [Point(lon, lat) for lat, lon in ordered_stop_coords_viz]
                 stops_gdf_viz = gpd.GeoDataFrame({'stop_id': valid_stop_ids_in_order_viz}, geometry=stop_geometries_viz, crs="EPSG:4326")

                 # --- Create Visualization ---
                 print("Creating map...")
                 try:
                     # Calculate center point for the map using union_all
                     map_center_viz = [stops_gdf_viz.geometry.union_all().centroid.y, stops_gdf_viz.geometry.union_all().centroid.x]
                 except Exception as center_e:
                     print(f"Warning: Could not calculate map center: {center_e}. Using first stop.")
                     # Check again stops_gdf_viz is not empty before accessing iloc[0]
                     if not stops_gdf_viz.empty:
                          map_center_viz = [stops_gdf_viz.iloc[0].geometry.y, stops_gdf_viz.iloc[0].geometry.x]
                     else:
                          map_center_viz = [28.6, 77.2] # Fallback center

                 m_viz = folium.Map(location=map_center_viz, zoom_start=13) # Adjust zoom as needed

                 # Add stops as markers
                 print(f"Adding {len(stops_gdf_viz)} stop markers...")
                 for idx, row in stops_gdf_viz.iterrows():
                      if row.geometry and not row.geometry.is_empty:
                          # Try to get original sequence number if possible
                          original_sequence = trip_stop_times_viz[trip_stop_times_viz['stop_id'] == row['stop_id']]['stop_sequence'].iloc[0] if not trip_stop_times_viz.empty else idx
                          folium.Marker(
                              location=[row.geometry.y, row.geometry.x],
                              popup=f"Stop ID: {row['stop_id']}<br>Seq: {original_sequence}", # Use original sequence
                              tooltip=f"Stop ID: {row['stop_id']}",
                              icon=folium.Icon(color='purple', icon='circle', prefix='fa') # Changed icon/color
                          ).add_to(m_viz)

                 # Add click for lat/lon
                 folium.LatLngPopup().add_to(m_viz)

                 legend_html = """
                    <div style="position: fixed;
                    bottom: 50px; left: 50px; width: 230px; height: 100px; /* Increased width */
                    border:2px solid grey; z-index:9999; font-size:14px;
                    background-color: white; padding: 5px;
                    ">
                    &nbsp;<b>Legend</b><br>
                    &nbsp;<i class="fa fa-bus" style="color:purple"></i>&nbsp; Bus Stop<br>
                    </div>"""

                # Then right after this, ADD THIS LINE to add the legend to the map:
                 m_viz.get_root().html.add_child(folium.Element(legend_html))

                 # Display the map directly in the output
                 print("Displaying map...")
                 display(m_viz)

            else:
                 print("No valid stops with coordinates found to display.")

    except Exception as e_viz:
        print(f"An error occurred during visualization preparation: {e_viz}")
        import traceback
        traceback.print_exc()
    finally:
        # Clean up temporary variables created in this cell
        del stops_gdf_viz
        if 'trip_stop_times_viz' in locals(): del trip_stop_times_viz
        if 'stops_info_dict_viz' in locals(): del stops_info_dict_viz
        if 'temp_stop_times' in locals(): del temp_stop_times
        gc.collect()
        print("Cleaned up visualization variables.")

--- Visualizing Stops for Trip ID: 362_13_36 ---
Found 94 stops in sequence for trip 362_13_36.
Found valid coordinates for 94 stops.
Creating map...
Adding 94 stop markers...
Displaying map...


Cleaned up visualization variables.


In [ ]:
# Cell 3: Process ALL Stops for Target Trip + Cleanup

import gc
import pandas as pd # Make sure pandas is available if run standalone
import geopandas as gpd # Make sure geopandas is available
from shapely.geometry import Point # Ensure Point is available
import os

# Make sure 'TARGET_TRIP_ID', 'stops_df', 'stop_times_df' exist from Cell 2 before running
if 'TARGET_TRIP_ID' not in locals() or 'stops_df' not in locals() or 'stop_times_df' not in locals():
    print("Error: Please run Cell 1 and Cell 2 successfully first.")
    stops_gdf = gpd.GeoDataFrame()
    raise NameError("Required variables from Cell 2 not found.")
else:
    print(f"\n--- Processing ALL Stops for Trip ID: {TARGET_TRIP_ID} ---")

    # Ensure stop_sequence is numeric for sorting
    try:
        stop_times_df['stop_sequence'] = pd.to_numeric(stop_times_df['stop_sequence'], errors='coerce')
        stop_times_df = stop_times_df.dropna(subset=['stop_sequence'])
        stop_times_df['stop_sequence'] = stop_times_df['stop_sequence'].astype(int)
    except KeyError: print("Error: 'stop_sequence' column not found."); raise
    except Exception as e: print(f"Warning: Could not process stop_sequence: {e}")

    # Filter for the target trip and sort
    trip_stop_times = stop_times_df[stop_times_df['trip_id'] == TARGET_TRIP_ID].sort_values('stop_sequence')

    # --- Cleanup: Delete original stop_times_df ---
    if 'stop_times_df' in locals(): del stop_times_df; gc.collect(); print("Cleaned up full stop_times_df.")
    # -------------------------------------------

    if trip_stop_times.empty:
        print(f"Error: No stop times found for trip_id '{TARGET_TRIP_ID}'.")
        stops_gdf = gpd.GeoDataFrame()
    else:
        # Get the FULL list of ordered stop IDs
        ordered_stop_ids = trip_stop_times['stop_id'].tolist()
        print(f"Using all {len(ordered_stop_ids)} stops for processing.")
        # --------------------------------

        if len(ordered_stop_ids) < 2:
             print(f"Error: Fewer than 2 stops found for this trip.")
             stops_gdf = gpd.GeoDataFrame()
        else:
            print(f"Full Stop sequence (first 10): {ordered_stop_ids[:10]}...")

            # --- Get Stop Coordinates (for the FULL list) ---
            if 'stops_df' in locals():
                trip_stops_info_filtered = stops_df[stops_df['stop_id'].isin(ordered_stop_ids)]
                del stops_df; gc.collect(); print("Cleaned up full stops_df.")

                stops_info_dict = trip_stops_info_filtered.set_index('stop_id').to_dict('index')
                del trip_stops_info_filtered; gc.collect(); print("Cleaned up trip_stops_info_filtered.")

                ordered_stop_coords = []
                valid_stop_ids_in_order = []
                # Iterate through the FULL list
                for sid in ordered_stop_ids:
                    stop_info = stops_info_dict.get(sid)
                    if stop_info and isinstance(stop_info.get('stop_lat'), (int, float)) and isinstance(stop_info.get('stop_lon'), (int, float)) and pd.notna(stop_info.get('stop_lat')) and pd.notna(stop_info.get('stop_lon')):
                        ordered_stop_coords.append((stop_info['stop_lat'], stop_info['stop_lon']))
                        valid_stop_ids_in_order.append(sid)
                    else: print(f"Warning: Skipping stop_id '{sid}' due to missing/invalid coordinates.")

                print(f"\nCoordinates extracted for {len(valid_stop_ids_in_order)} valid stops.")
                del stops_info_dict; gc.collect(); print("Cleaned up stops_info_dict.")

                if not valid_stop_ids_in_order or len(valid_stop_ids_in_order) < 2:
                    print("Error: Fewer than 2 valid stops remain after coordinate check.")
                    stops_gdf = gpd.GeoDataFrame()
                else:
                    try:
                         stop_geometries = [Point(lon, lat) for lat, lon in ordered_stop_coords]
                         # Create stops_gdf containing ALL the trip's valid stops
                         stops_gdf = gpd.GeoDataFrame({'stop_id': valid_stop_ids_in_order}, geometry=stop_geometries, crs="EPSG:4326")
                         print("Created stops_gdf for route (first 5 rows):")
                         print(stops_gdf.head())
                    except Exception as e:
                         print(f"Error creating GeoDataFrame for stops: {e}")
                         stops_gdf = gpd.GeoDataFrame()
            else:
                 print("Error: stops_df not found from Cell 2.")
                 stops_gdf = gpd.GeoDataFrame()

# Ensure stops_gdf is defined for later cells
if 'stops_gdf' not in locals():
    stops_gdf = gpd.GeoDataFrame()

# --- Debug block (Optional but useful) ---
print("\n--- Debugging Processed Route Stops Data ---")
if 'stops_gdf' in locals() and not stops_gdf.empty:
    print(f"Final count of stops in stops_gdf: {len(stops_gdf)}")
    # ... (rest of debug block if you want it) ...
    try:
        dbg_min_lon, dbg_min_lat, dbg_max_lon, dbg_max_lat = stops_gdf.total_bounds
        print(f"Calculated Bounding Box for ALL {len(stops_gdf)} stops:")
        print(f"  North: {dbg_max_lat}, South: {dbg_min_lat}, East: {dbg_max_lon}, West: {dbg_min_lon}")
    except Exception as e: print(f"Could not calculate bounds for debugging: {e}")
else: print("stops_gdf is empty or not defined, cannot debug.")
print("--- End Debugging Processed Route Stops Data ---")

print("Cell 3/5: Full route stop processing complete.")


--- Processing ALL Stops for Trip ID: 362_13_36 ---
Cleaned up full stop_times_df.
Using all 94 stops for processing.
Full Stop sequence (first 10): ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']...
Cleaned up full stops_df.
Cleaned up trip_stops_info_filtered.

Coordinates extracted for 94 valid stops.
Cleaned up stops_info_dict.
Created stops_gdf for route (first 5 rows):
  stop_id                   geometry
0       0   POINT (77.0881 28.85196)
1       1  POINT (77.08855 28.85331)
2       2  POINT (77.08932 28.85497)
3       3  POINT (77.09285 28.85771)
4       4  POINT (77.09711 28.85768)

--- Debugging Processed Route Stops Data ---
Final count of stops in stops_gdf: 94
Calculated Bounding Box for ALL 94 stops:
  North: 28.85771369934082, South: 28.610319137573242, East: 77.10202026367188, West: 76.96136474609375
--- End Debugging Processed Route Stops Data ---
Cell 3/5: Full route stop processing complete.


In [ ]:
!apt-get update
!apt-get install -y osmium-tool
# Cell 1 Add-on / Prerequisite: Install branca if needed
!pip install branca

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,154 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,543 kB]
Get:13 https://developer.download.nvidia.com/com

In [ ]:
# Cell 4: Clip OSM PBF Extract to Segment Bounding Box

import os
import sys # Import sys for flush

print("\n--- Clipping PBF file to Segment Area ---")

# --- Prerequisites Check ---
if 'stops_gdf' not in locals() or stops_gdf.empty:
    print("Error: 'stops_gdf' (for the segment) not found or empty. Please run Cell 3 successfully first.")
    clipped_osm_filepath = None # Indicate failure
else:
    # --- Define Input/Output Files ---
    full_osm_filepath = '/content/national_capital_territory_of_delhi-latest.osm.pbf'
    # Define name for the smaller, clipped output file
    clipped_osm_filepath = '/content/delhi_segment_clipped.osm.pbf'
    # ---------------------------------

    if not os.path.exists(full_osm_filepath):
        print(f"Error: Full PBF file not found at {full_osm_filepath}. Please ensure it's uploaded.")
        clipped_osm_filepath = None # Indicate failure
    else:
        try:
            # 1. Calculate Bounding Box from segment's stops_gdf
            min_lon, min_lat, max_lon, max_lat = stops_gdf.total_bounds
            buffer = 0.01 # Add a small buffer (adjust if needed)
            west, south, east, north = min_lon - buffer, min_lat - buffer, max_lon + buffer, max_lat + buffer
            print(f"Calculated BBox for clipping: W={west:.6f}, S={south:.6f}, E={east:.6f}, N={north:.6f}")

            # 2. Construct the osmium command
            # Use --strategy complete_ways to try and include nodes for ways crossing boundary
            # Use --overwrite to allow re-running the cell
            osmium_command = f"osmium extract --bbox {west},{south},{east},{north} \"{full_osm_filepath}\" -o \"{clipped_osm_filepath}\" --strategy complete_ways --overwrite"
            print(f"Running osmium command:\n {osmium_command}")

            # 3. Execute the command using system call
            exit_code = os.system(osmium_command)
            sys.stdout.flush() # Make sure output is shown
            sys.stderr.flush()

            # 4. Check result
            if exit_code == 0 and os.path.exists(clipped_osm_filepath):
                print(f"Successfully created clipped PBF file: {clipped_osm_filepath}")
                # Optionally check file size
                file_size = os.path.getsize(clipped_osm_filepath)
                print(f"Clipped file size: {file_size / (1024*1024):.2f} MB")
            else:
                print(f"Error: Osmium command failed with exit code {exit_code} or output file not found.")
                clipped_osm_filepath = None # Indicate failure

        except Exception as e:
            print(f"An error occurred during PBF clipping: {e}")
            clipped_osm_filepath = None # Indicate failure

# Ensure variable exists for next cell check
if 'clipped_osm_filepath' not in locals():
     clipped_osm_filepath = None

print("Cell 4: PBF clipping attempt complete.")
# Output variable: 'clipped_osm_filepath' (path to the small PBF, or None if failed)


--- Clipping PBF file to Segment Area ---
Calculated BBox for clipping: W=76.951365, S=28.600319, E=77.112020, N=28.867714
Running osmium command:
 osmium extract --bbox 76.95136474609374,28.60031913757324,77.11202026367188,28.867713699340822 "/content/national_capital_territory_of_delhi-latest.osm.pbf" -o "/content/delhi_segment_clipped.osm.pbf" --strategy complete_ways --overwrite
Successfully created clipped PBF file: /content/delhi_segment_clipped.osm.pbf
Clipped file size: 4.47 MB
Cell 4: PBF clipping attempt complete.


In [ ]:
# Cell 5: Load Clipped Graph using Pyrosm, Generate Path, AND SAVE GRAPH (+ Cleanup)

import time
import gc
import osmnx as ox
import networkx as nx
from shapely.geometry import LineString, Point
import pandas as pd
import geopandas as gpd
import os # For checking file path later if needed
try:
    from pyrosm import OSM, get_data
except ImportError: print("Error: pyrosm library not found."); raise

print("\n--- Loading Clipped Graph using Pyrosm & Generating Path ---")
standard_path_geometry = None # Initialize path variable
G = None # Initialize graph variable
graph_filepath_segment = None # Initialize variable for saved graph path

# --- Prerequisites Check ---
# Check for the clipped file path from Cell 3.5 and stops_gdf from Cell 3
if 'clipped_osm_filepath' not in locals() or not clipped_osm_filepath:
     print("Error: Clipped OSM file path not generated successfully in previous step (Cell 3.5).")
elif 'stops_gdf' not in locals() or stops_gdf.empty:
    print("Error: 'stops_gdf' (for the segment) not found or empty (Cell 3).")
elif len(stops_gdf) < 2:
    print("Error: Fewer than 2 stops in segment's stops_gdf (Cell 3).")
# --------------------------

else: # Proceed only if prerequisites seem okay
    osm_file_path = clipped_osm_filepath # Use the path to the clipped file

    try:
        # 1. Initialize Pyrosm OSM reader object for the CLIPPED file
        print(f"Initializing pyrosm reader for clipped file: {osm_file_path} ...")
        osm_reader = OSM(osm_file_path)
        print("Pyrosm reader initialized.")

        # 2. Extract the drivable network using Pyrosm (from small file)
        print("Extracting 'drive' network with pyrosm...")
        start_time_extract = time.time()
        nodes, edges = osm_reader.get_network(network_type="driving", nodes=True)
        end_time_extract = time.time()
        print(f"Extracted network nodes ({len(nodes) if nodes is not None else 0}) and edges ({len(edges) if edges is not None else 0}) in {end_time_extract - start_time_extract:.2f} seconds.")

        if nodes is None or edges is None or nodes.empty or edges.empty:
             raise ValueError("Pyrosm failed to extract nodes or edges from clipped file.")

        # 3. Convert Pyrosm network to NetworkX graph (should use less RAM now)
        print("Converting pyrosm network to NetworkX graph...")
        start_time_graph = time.time()
        G = osm_reader.to_graph(nodes, edges, graph_type="networkx")
        end_time_graph = time.time()
        print(f"Converted to NetworkX graph in {end_time_graph - start_time_graph:.2f} seconds.")
        print(f"Graph contains {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

        # Cleanup intermediate pyrosm variables
        del osm_reader, nodes, edges
        gc.collect()
        print("Cleaned up pyrosm reader, nodes, edges.")

        # --- Steps 4, 5, 6: Find nodes, route, create geometry ---
        # (These remain the same as the previous working version)

        # Ensure stops_gdf CRS matches graph (Assume EPSG:4326 if not set)
        graph_crs = G.graph.get('crs', "EPSG:4326")
        if stops_gdf.crs != graph_crs:
             stops_gdf = stops_gdf.to_crs(graph_crs)

        # Ensure graph nodes have x/y attributes
        sample_node = list(G.nodes())[0]
        if 'x' not in G.nodes[sample_node] or 'y' not in G.nodes[sample_node]:
             print("Adding 'x'/'y' attributes from node geometry...")
             missing_geom_count = 0
             for node_id, data in G.nodes(data=True):
                  if 'geometry' in data and isinstance(data['geometry'], Point):
                       G.nodes[node_id]['x'] = data['geometry'].x
                       G.nodes[node_id]['y'] = data['geometry'].y
                  else: missing_geom_count += 1
             if missing_geom_count > 0: print(f"Warning: {missing_geom_count} nodes missing geometry.")

        # 4. Get nearest OSM nodes
        stop_nodes = ox.nearest_nodes(G, X=stops_gdf.geometry.x.tolist(), Y=stops_gdf.geometry.y.tolist())

        # 5. Calculate shortest path
        route_nodes = []
        print("Calculating shortest paths between consecutive stops...")
        start_time_routing = time.time()
        for i in range(len(stop_nodes) - 1):
            start_node, end_node = stop_nodes[i], stop_nodes[i+1]
            if start_node == end_node: continue
            try:
                path_segment = nx.shortest_path(G, start_node, end_node, weight='length')
                if path_segment:
                    if not route_nodes or path_segment[0] != route_nodes[-1]: route_nodes.extend(path_segment)
                    else: route_nodes.extend(path_segment[1:])
            except nx.NetworkXNoPath: print(f"Warning: No path found (nx) for segment {i}...")
            except Exception as path_e: print(f"Warning: Error finding path segment {i}: {path_e}")
        end_time_routing = time.time()
        print(f"Finished calculating path segments in {end_time_routing - start_time_routing:.2f} seconds.")

        # 6. Create final geometry
        if route_nodes:
            print(f"Generated route for segment with {len(route_nodes)} nodes.")
            points = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in route_nodes if node in G.nodes]
            if len(points) >= 2: standard_path_geometry = LineString(points); print("Created final standard_path_geometry LineString.")
            else: print("Warning: Fewer than 2 valid points for final LineString.")
        else: print("Error: Could not generate any route nodes.")

        # --- 7. SAVE the graph for THIS SEGMENT ONLY ---
        if G is not None:
             # Use TARGET_TRIP_ID and maybe segment indices in filename
             # Ensure these variables exist from Cell 2 / Cell 3
             trip_id_for_file = TARGET_TRIP_ID if 'TARGET_TRIP_ID' in locals() else "UNKNOWN_TRIP"
             start_idx = SEGMENT_START_SEQUENCE_IDX if 'SEGMENT_START_SEQUENCE_IDX' in locals() else 'start'
             end_idx = SEGMENT_END_SEQUENCE_IDX if 'SEGMENT_END_SEQUENCE_IDX' in locals() else 'end'
             graph_filepath_segment = f"/content/graph_{trip_id_for_file}_seg_{start_idx}-{end_idx}.graphml"

             print(f"\nSaving segment graph G to {graph_filepath_segment}...")
             start_time_save = time.time()
             ox.io.save_graphml(G, filepath=graph_filepath_segment)
             end_time_save = time.time()
             print(f"Segment graph saved successfully in {end_time_save - start_time_save:.2f} seconds.")
        else:
             print("Skipping graph save because G is None.")
        # --------------------------------------------------

    except FileNotFoundError: print(f"Error: Clipped OSM file not found at '{osm_file_path}'. Check Cell 3.5 output.")
    except ImportError: print("Error: Could not import pyrosm.")
    except Exception as e: print(f"Error during graph loading/path generation from clipped file: {e}"); import traceback; traceback.print_exc()
    finally:
        # Cleanup: Delete Graph from memory (AFTER saving)
        if 'G' in locals() and G is not None: del G; gc.collect(); print("Cleaned up NetworkX graph object.")
        else: print("Graph object G was not created or already deleted.")

# else block from prerequisite check finishes here
# else:
#    print("Skipping path generation because prerequisites not met.")

print("Cell 4/5: Path generation attempt using clipped PBF complete.")
# Output variables: 'standard_path_geometry', 'graph_filepath_segment'


--- Loading Clipped Graph using Pyrosm & Generating Path ---
Initializing pyrosm reader for clipped file: /content/delhi_segment_clipped.osm.pbf ...
Pyrosm reader initialized.
Extracting 'drive' network with pyrosm...
Extracted network nodes (174386) and edges (209569) in 11.18 seconds.
Converting pyrosm network to NetworkX graph...
Converted to NetworkX graph in 19.31 seconds.
Graph contains 171857 nodes and 396956 edges.
Cleaned up pyrosm reader, nodes, edges.
Calculating shortest paths between consecutive stops...
Finished calculating path segments in 0.09 seconds.
Generated route for segment with 1634 nodes.
Created final standard_path_geometry LineString.

Saving segment graph G to /content/graph_362_13_36_seg_start-end.graphml...
Segment graph saved successfully in 107.99 seconds.
Cleaned up NetworkX graph object.
Cell 4/5: Path generation attempt using clipped PBF complete.


In [ ]:
# Cell: Visualize Standard Path (Path A) for Full Route
# Purpose: Load the graph from the clipped PBF for the full route,
#          calculate the shortest path between the first and last stop,
#          and visualize this path in green along with the stops.
# Assumes Cell 1, 2, 3, and 3.5 have run successfully.

import osmnx as ox
import networkx as nx
import geopandas as gpd
import folium
import time
import gc
import os
import traceback
from shapely.geometry import Point, LineString # For node attribute check
try:
    from pyrosm import OSM # Requires pyrosm installation
except ImportError:
    print("ERROR: pyrosm library not found. Please install it: !pip install pyrosm")
    raise

print("\n--- Visualizing Standard Path (Path A) for Full Route ---")

# --- Configuration ---
# !!! Ensure this matches the TARGET_TRIP_ID used in Cells 2, 3, 3.5 !!!
TARGET_TRIP_ID_VIZ = '362_13_36'
# Path to the CLIPPED PBF file created by Cell 3.5 for the FULL trip
# !!! Adjust filename if Cell 3.5 saved it differently !!!
clipped_osm_filepath_viz = f'/content/delhi_segment_clipped.osm.pbf'
# --------------------

# Initialize variables
G_viz = None
standard_route_nodes = None
standard_route_geom = None
map_viz_standard = None

# --- Prerequisites Check ---
if 'stops_gdf' not in locals() or stops_gdf.empty: # stops_gdf for the FULL route from Cell 3
    print("Error: 'stops_gdf' (for the full route) not found or empty. Please run Cell 3 successfully first.")
elif not os.path.exists(clipped_osm_filepath_viz):
     print(f"Error: Clipped PBF file {clipped_osm_filepath_viz} not found. Please run Cell 3.5 successfully first.")
# -----------------------------

else: # Proceed only if prerequisites seem okay
    try:
        # === Part 1: Load Graph for Route from Clipped PBF ===
        print(f"Loading graph from clipped PBF: {clipped_osm_filepath_viz} ...")
        start_time_load = time.time()
        osm_reader_viz = OSM(clipped_osm_filepath_viz)
        nodes_viz, edges_viz = osm_reader_viz.get_network(network_type="driving", nodes=True)
        if nodes_viz is None or edges_viz is None or nodes_viz.empty or edges_viz.empty:
            raise ValueError("Pyrosm failed to extract nodes/edges from clipped file.")
        print("Converting pyrosm network to NetworkX graph...")
        G_viz = osm_reader_viz.to_graph(nodes_viz, edges_viz, graph_type="networkx")
        del osm_reader_viz, nodes_viz, edges_viz; gc.collect()
        end_time_load = time.time()
        print(f"Loaded graph G_viz in {end_time_load - start_time_load:.2f} seconds ({G_viz.number_of_nodes()} nodes, {G_viz.number_of_edges()} edges).")

        # Ensure graph/stops CRS and node attributes are okay
        graph_crs_viz = G_viz.graph.get('crs', "EPSG:4326")
        if stops_gdf.crs != graph_crs_viz:
            stops_gdf_viz_proj = stops_gdf.to_crs(graph_crs_viz)
        else:
            stops_gdf_viz_proj = stops_gdf.copy() # Use a copy

        sample_node_viz = list(G_viz.nodes())[0]
        if 'x' not in G_viz.nodes[sample_node_viz] or 'y' not in G_viz.nodes[sample_node_viz]:
            print("Adding 'x'/'y' attributes from node geometry...")
            for node_id, data in G_viz.nodes(data=True):
                 if 'geometry' in data and isinstance(data['geometry'], Point):
                     G_viz.nodes[node_id]['x'], G_viz.nodes[node_id]['y'] = data['geometry'].x, data['geometry'].y

        # === Part 2: Calculate Standard Shortest Path ===
        print("\nCalculating shortest path between first and last stop...")
        # Get first and last stop coordinates from the full route stops_gdf
        first_stop_geom = stops_gdf_viz_proj.iloc[0].geometry
        last_stop_geom = stops_gdf_viz_proj.iloc[-1].geometry

        # Find nearest nodes on the graph G_viz
        orig_node = ox.nearest_nodes(G_viz, X=first_stop_geom.x, Y=first_stop_geom.y)
        dest_node = ox.nearest_nodes(G_viz, X=last_stop_geom.x, Y=last_stop_geom.y)
        print(f"  Origin Node: {orig_node}, Destination Node: {dest_node}")

        # Calculate the shortest path based on 'length' attribute
        standard_route_nodes = nx.shortest_path(G_viz, source=orig_node, target=dest_node, weight='length')
        print(f"  Found shortest path with {len(standard_route_nodes)} nodes.")

        # === Part 3: Create Visualization ===
        print("Creating map...")
        try:
             map_center_viz = [stops_gdf_viz_proj.geometry.union_all().centroid.y, stops_gdf_viz_proj.geometry.union_all().centroid.x]
        except Exception as center_e:
             print(f"Warning: Could not calculate map center: {center_e}. Using first stop.")
             map_center_viz = [first_stop_geom.y, first_stop_geom.x]

        map_viz_standard = folium.Map(location=map_center_viz, zoom_start=12) # Adjust zoom as needed

        # Plot the Standard Path (Green)
        if standard_route_nodes and len(standard_route_nodes) >= 2:
            points_viz = [(G_viz.nodes[node]['y'], G_viz.nodes[node]['x']) for node in standard_route_nodes if node in G_viz.nodes]
            if len(points_viz) >= 2:
                folium.PolyLine(
                    points_viz,
                    color='green',
                    weight=5,
                    opacity=0.8,
                    tooltip='Standard Shortest Path (A)',
                    name='Standard Path (A)'
                ).add_to(map_viz_standard)
                print("Added standard shortest path (green) to map.")
            else:
                print("Warning: Could not extract coordinates for standard path.")
        else:
            print("Warning: Standard shortest path calculation failed or resulted in < 2 nodes.")

        # Plot Stops (Red Markers)
        print(f"Adding {len(stops_gdf_viz_proj)} stop markers...")
        for idx, row in stops_gdf_viz_proj.iterrows():
              if row.geometry and not row.geometry.is_empty:
                  folium.Marker(
                      location=[row.geometry.y, row.geometry.x],
                      popup=f"Stop ID: {row['stop_id']}<br>Seq Index: {idx}", # Use index as sequence proxy
                      tooltip=f"Stop ID: {row['stop_id']}",
                      icon=folium.Icon(color='red', icon='bus', prefix='fa')
                  ).add_to(map_viz_standard)

        # Add Legend
        legend_html = """
             <div style="position: fixed; bottom: 50px; left: 50px; width: 200px; height: 60px;
                         border:2px solid grey; z-index:9999; font-size:14px;
                         background-color: white; padding: 5px;">
             &nbsp;<b>Legend</b><br>
             &nbsp;<i class="fa fa-bus" style="color:red"></i>&nbsp; Bus Stop<br>
             &nbsp;<span style="background-color: green; display: inline-block; width: 20px; height: 4px; vertical-align: middle;"></span>&nbsp; Standard Shortest Path<br>
             </div> """
        legend = branca.element.Element(legend_html)
        map_viz_standard.get_root().html.add_child(legend)

        # Add Layer Control
        folium.LayerControl().add_to(map_viz_standard)

        # Save and display
        map_filename_standard = f"/content/standard_path_map_{TARGET_TRIP_ID_VIZ}.html"
        map_viz_standard.save(map_filename_standard)
        print(f"\nSaved standard path map to {map_filename_standard}.")
        print("Displaying map...")
        display(map_viz_standard)

    except FileNotFoundError: print(f"Error: Clipped PBF file {clipped_osm_filepath_viz} not found.")
    except ImportError: print("Error: Could not import required library (e.g., pyrosm).")
    except nx.NetworkXNoPath: print("Error: No path found between the start and end nodes on the graph.")
    except Exception as e_viz: print(f"An error occurred: {e_viz}"); traceback.print_exc()
    finally:
        # Clean up graph object created in this cell
        if 'G_viz' in locals() and G_viz is not None: del G_viz; gc.collect(); print("Cleaned up visualization graph object.")

# else block from prerequisite check finishes here
# else:
#    print("Skipping standard path visualization: Prerequisites not met.")

print("\nCell: Standard Path Visualization complete.")


# **Explanation:**

# 1.  **Configuration:** Sets the `TARGET_TRIP_ID_VIZ` and the path to the clipped PBF file generated in Cell 3.5. **Make sure these match your actual trip ID and filename.**
# 2.  **Prerequisites:** Checks that `stops_gdf` (from Cell 3) exists and the clipped PBF file exists.
# 3.  **Load Graph:** Loads the graph `G_viz` from the clipped PBF file using `pyrosm`.
# 4.  **Find Start/End Nodes:** Gets the coordinates of the first and last stop from `stops_gdf` and finds the nearest nodes (`orig_node`, `dest_node`) on the graph `G_viz` using `osmnx.nearest_nodes`.
# 5.  **Calculate Shortest Path:** Uses `networkx.shortest_path` with `weight='length'` to find the node list (`standard_route_nodes`) representing the geometrically shortest path between the origin and destination nodes on the graph.
# 6.  **Visualize:**
#     * Creates a `folium` map centered appropriately.
#     * Extracts the coordinates for the `standard_route_nodes`.
#     * Plots this path as a **green** `PolyLine` on the map.
#     * Plots the bus stops from `stops_gdf` as red bus icons.
#     * Adds a legend.
#     * Saves and displays the map.

# This cell will show you the route that the `shortest_path` algorithm considers to be the standard "Path A" for the entire segment, based purely on minimizing distance on the OSM road network. You can compare this to the actual operational route (if known) and the path chosen by the PSO in later visualizatio
# '''


--- Visualizing Standard Path (Path A) for Full Route ---
Loading graph from clipped PBF: /content/delhi_segment_clipped.osm.pbf ...
Converting pyrosm network to NetworkX graph...
Loaded graph G_viz in 33.24 seconds (171857 nodes, 396956 edges).

Calculating shortest path between first and last stop...
  Origin Node: 9908952639, Destination Node: 2283797435
  Found shortest path with 725 nodes.
Creating map...
Added standard shortest path (green) to map.
Adding 94 stop markers...
An error occurred: name 'branca' is not defined
Cleaned up visualization graph object.

Cell: Standard Path Visualization complete.


Traceback (most recent call last):
  File "<ipython-input-7-d468e4e16f47>", line 137, in <cell line: 0>
    legend = branca.element.Element(legend_html)
             ^^^^^^
NameError: name 'branca' is not defined


In [ ]:
# Cell 6: Load Clipped Segment Graph & Find Alternatives

import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, LineString
import pandas as pd
import time
import gc
import pickle # Using pickle to save the complex dictionary
import os # For checking file path
import traceback # For detailed errors
try:
    from pyrosm import OSM
except ImportError: print("Error: pyrosm library not found."); raise

print("\n--- Loading Clipped Segment Graph & Finding Alternatives ---")

# --- Define the Clipped PBF Filepath ---
# This is the SMALL PBF file created by Cell 3.5
clipped_osm_filepath = '/content/delhi_segment_clipped.osm.pbf' # Make sure this file exists
# ---------------------------------------

# Initialize results structure
alternative_paths_data = {}
G = None # Initialize graph variable

# --- Prerequisites Check ---
if 'stops_gdf' not in locals() or stops_gdf.empty: # stops_gdf for the segment from Cell 3
    print("Error: 'stops_gdf' (for the segment) not found or empty. Please run Cell 3 successfully first.")
elif not isinstance(clipped_osm_filepath, str) or not os.path.exists(clipped_osm_filepath):
     print(f"Error: Clipped PBF file {clipped_osm_filepath} not found. Please run Cell 3.5 successfully first.")
elif 'TARGET_TRIP_ID' not in locals():
     print("Warning: TARGET_TRIP_ID not defined. Using default for output filenames.")
     TARGET_TRIP_ID = "UNKNOWN_TRIP_SEG" # Default for segment
# -----------------------------

else: # Proceed only if prerequisites seem okay
    try:
        # === Part 1: Load Graph for Segment from Clipped PBF ===
        osm_file_path = clipped_osm_filepath
        print(f"Initializing pyrosm reader for clipped file: {osm_file_path} ...")
        osm_reader = OSM(osm_file_path)
        print("Pyrosm reader initialized.")

        print("Extracting 'drive' network with pyrosm...")
        start_time_extract = time.time()
        nodes, edges = osm_reader.get_network(network_type="driving", nodes=True)
        end_time_extract = time.time()
        print(f"Extracted network nodes/edges in {end_time_extract - start_time_extract:.2f} seconds.")

        if nodes is None or edges is None or nodes.empty or edges.empty:
             raise ValueError("Pyrosm failed to extract nodes or edges from clipped file.")

        print("Converting pyrosm network to NetworkX graph...")
        start_time_graph = time.time()
        G = osm_reader.to_graph(nodes, edges, graph_type="networkx")
        end_time_graph = time.time()
        print(f"Converted to NetworkX graph in {end_time_graph - start_time_graph:.2f} seconds.")
        print(f"Segment graph contains {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

        del osm_reader, nodes, edges # Clean up intermediate pyrosm vars
        gc.collect()
        print("Cleaned up pyrosm reader, nodes, edges.")

        # === Part 2: Find Alternative Paths using the loaded graph G ===

        graph_crs = G.graph.get('crs', "EPSG:4326")
        if stops_gdf.crs != graph_crs: stops_gdf = stops_gdf.to_crs(graph_crs)

        # Ensure nodes have x/y attributes
        sample_node = list(G.nodes())[0]
        if 'x' not in G.nodes[sample_node] or 'y' not in G.nodes[sample_node]:
            print("Adding 'x'/'y' attributes from node geometry...")
            missing_geom_count = 0
            for node_id, data in G.nodes(data=True):
                 if 'geometry' in data and isinstance(data['geometry'], Point):
                      G.nodes[node_id]['x'] = data['geometry'].x; G.nodes[node_id]['y'] = data['geometry'].y
                 else: missing_geom_count += 1
            if missing_geom_count > 0: print(f"Warning: {missing_geom_count} nodes missing geometry.")

        # Define DP and Rejoin Point Coordinates (Lat, Lon)
        # Using the coordinates from your corrected clarification
        main_dp1_coord = (28.7092, 76.9822); int_dp1a_coord = (28.7100, 76.9819); int_dp1b_coord = (28.7107, 76.9807); rejoin_p1_coord = (28.7104, 76.9806)
        main_dp2_coord = (28.7172, 76.9625); int_dp2a_coord = (28.7167, 76.9620); int_dp2b_coord = (28.7171, 76.9609); int_dp2c_coord = (28.7177, 76.9611); rejoin_p2_coord = (28.7178, 76.9613)
        main_dp3_coord = (28.7280, 76.9666); int_dp3a_coord = (28.7301, 76.9677); rejoin_p3_coord = (28.7301, 76.9665)

        main_dp_coords_list = [main_dp1_coord, main_dp2_coord, main_dp3_coord]
        intermediate_dp_coords_by_main_dp_index = { 0: [int_dp1a_coord, int_dp1b_coord], 1: [int_dp2a_coord, int_dp2b_coord, int_dp2c_coord], 2: [int_dp3a_coord] }
        rejoin_point_coords_by_main_dp_index = { 0: rejoin_p1_coord, 1: rejoin_p2_coord, 2: rejoin_p3_coord }
        print(f"Defined {len(main_dp_coords_list)} Main DPs and their alternative path structures.")

        # Find nearest nodes for all necessary points
        print("Finding nearest nodes for DPs and Rejoin Points...")
        all_coords_dict = {}; node_lookup = {}
        for i, coord in enumerate(main_dp_coords_list): all_coords_dict[f'main_{i}'] = coord
        for i, coord_list in intermediate_dp_coords_by_main_dp_index.items():
             for j, coord in enumerate(coord_list): all_coords_dict[f'int_{i}_{j}'] = coord
        for i, coord in rejoin_point_coords_by_main_dp_index.items(): all_coords_dict[f'rejoin_{i}'] = coord
        coord_labels = list(all_coords_dict.keys()); all_lats = [c[0] for c in all_coords_dict.values()]; all_lons = [c[1] for c in all_coords_dict.values()]
        all_nodes_list = ox.nearest_nodes(G, X=all_lons, Y=all_lats)
        node_lookup = {label: node for label, node in zip(coord_labels, all_nodes_list)}
        print("Nodes found.")

        # Loop Through Defined Alternative Segments
        print(f"\nCalculating paths for {len(main_dp_coords_list)} alternative segments...")
        for i in range(len(main_dp_coords_list)):
            start_node = node_lookup.get(f'main_{i}')
            end_node = node_lookup.get(f'rejoin_{i}')
            if start_node is None or end_node is None: print(f"-- Skipping Segment {i+1}: Nodes not found."); continue
            print(f"-- Segment {i+1} (Main DP Node {start_node} -> RP Node {end_node}) --")
            alternative_paths_data[start_node] = {'end_node': end_node}
            if start_node == end_node: print("   Skipping: Start == End."); continue

            # Calculate Path A
            try:
                path_a_nodes = nx.shortest_path(G, start_node, end_node, weight='length')
                path_a_length = nx.path_weight(G, path_a_nodes, weight='length')
                alternative_paths_data[start_node]['A_nodes'] = path_a_nodes; alternative_paths_data[start_node]['A_length'] = path_a_length
                print(f"   Path A (DP{i+1} -> RP{i+1}): Found {len(path_a_nodes)} nodes, length {path_a_length:.2f}m")
            except Exception as e_a: print(f"   Path A (DP{i+1} -> RP{i+1}): Error - {e_a}")

            # Calculate Path B
            if i in intermediate_dp_coords_by_main_dp_index:
                inter_nodes = [node_lookup.get(f'int_{i}_{j}') for j in range(len(intermediate_dp_coords_by_main_dp_index[i]))]
                if None in inter_nodes: print(f"   Path B: Intermediate nodes not found."); continue
                via_nodes_b = [start_node] + inter_nodes + [end_node]; path_b_nodes = []; path_b_length = 0.0; possible_b = True
                print(f"   Calculating Path B via nodes: {via_nodes_b}")
                for j in range(len(via_nodes_b) - 1):
                     u, v = via_nodes_b[j], via_nodes_b[j+1];
                     if u == v: continue
                     try:
                         sub_segment = nx.shortest_path(G, u, v, weight='length')
                         path_b_length += nx.path_weight(G, sub_segment, weight='length')
                         if not path_b_nodes or sub_segment[0] != path_b_nodes[-1]: path_b_nodes.extend(sub_segment)
                         else: path_b_nodes.extend(sub_segment[1:])
                     except Exception as e_b_sub: print(f"     Warning: Sub-path error B ({u} -> {v}): {e_b_sub}"); possible_b = False; break
                if possible_b and path_b_nodes:
                     alternative_paths_data[start_node]['B_nodes'] = path_b_nodes; alternative_paths_data[start_node]['B_length'] = path_b_length
                     print(f"   Path B: Found {len(path_b_nodes)} nodes, length {path_b_length:.2f}m")
                else: print(f"   Path B: Could not be fully generated.")
            else: print(f"   Path B: No intermediate DPs defined.")

        # --- End of Segment Processing ---

        # --- Save the results ---
        trip_id_for_file = TARGET_TRIP_ID if 'TARGET_TRIP_ID' in locals() else "UNKNOWN_TRIP_SEG"
        start_idx = SEGMENT_START_SEQUENCE_IDX if 'SEGMENT_START_SEQUENCE_IDX' in locals() else 'start' # Requires these from Cell 3
        end_idx = SEGMENT_END_SEQUENCE_IDX if 'SEGMENT_END_SEQUENCE_IDX' in locals() else 'end'       # Requires these from Cell 3
        output_filepath = f"/content/alt_paths_{trip_id_for_file}.pkl"
        print(f"\nSaving alternative path data to {output_filepath}...")
        # The line with the syntax error:
        with open(output_filepath, 'wb') as f: # Added 'as' here
             pickle.dump(alternative_paths_data, f)
        print("Alternative path data saved successfully using pickle.")


    except FileNotFoundError: print(f"Error: Clipped PBF file {clipped_osm_filepath} not found.")
    except ImportError: print("Error: Could not import required library (e.g., pyrosm, osmnx).")
    except Exception as e_cell6: print(f"An error occurred: {e_cell6}"); traceback.print_exc()
    finally:
        # Cleanup loaded graph
        if 'G' in locals() and G is not None: del G; gc.collect(); print("Cleaned up graph object.")
        else: print("Graph object G was not created or already deleted.")

# else block from prerequisite check finishes here
# else:
#    print("Skipping alternative path finding: Prerequisites not met.")

print("Cell 6/6: Alternative path finding complete.")
# Output variable: 'alternative_paths_data' dictionary (also saved to pickle file)


--- Loading Clipped Segment Graph & Finding Alternatives ---
Initializing pyrosm reader for clipped file: /content/delhi_segment_clipped.osm.pbf ...
Pyrosm reader initialized.
Extracting 'drive' network with pyrosm...
Extracted network nodes/edges in 13.88 seconds.
Converting pyrosm network to NetworkX graph...
Converted to NetworkX graph in 20.11 seconds.
Segment graph contains 171857 nodes and 396956 edges.
Cleaned up pyrosm reader, nodes, edges.
Defined 3 Main DPs and their alternative path structures.
Finding nearest nodes for DPs and Rejoin Points...
Nodes found.

Calculating paths for 3 alternative segments...
-- Segment 1 (Main DP Node 9884263961 -> RP Node 3807950343) --
   Path A (DP1 -> RP1): Found 11 nodes, length 275.03m
   Calculating Path B via nodes: [np.int64(9884263961), np.int64(9884237589), np.int64(9884237580), np.int64(3807950343)]
   Path B: Found 22 nodes, length 313.43m
-- Segment 2 (Main DP Node 4223744551 -> RP Node 9889491412) --
   Path A (DP2 -> RP2): Foun

In [ ]:
# Cell 7: Load Data, Generate Geometries, Visualize with Legend

import osmnx as ox
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, LineString
import pandas as pd
import time
import gc
import pickle
import folium
from IPython.display import display
import os
import traceback # For detailed errors
import branca.element # For creating the HTML legend

try:
    from pyrosm import OSM
except ImportError: print("Error: pyrosm library not found."); raise

print("\n--- Loading Data, Generating Geometries, and Visualizing ---")

# --- Define Input Filepaths ---
# Ensure these match the outputs from Cell 3, Cell 3.5, and Cell 6
# Make sure TARGET_TRIP_ID matches the data processed
TARGET_TRIP_ID = '362_13_36' # Example: Make sure this is correct
SEGMENT_START_SEQUENCE_IDX = 44 # Make sure these match Cell 3 settings
SEGMENT_END_SEQUENCE_IDX = 50   # Make sure these match Cell 3 settings

clipped_osm_filepath = f'/content/delhi_segment_clipped.osm.pbf' # From Cell 3.5
alt_paths_filepath = f'/content/alt_paths_{TARGET_TRIP_ID}.pkl' # From Cell 6
# -----------------------------

# Initialize variables
G = None
alternative_paths_data = None
map_viz = None
standard_path_geometry_viz = None # Initialize
alternative_geometries_viz = [] # Initialize

# --- Prerequisites Check ---
if 'stops_gdf' not in locals() or stops_gdf.empty: # stops_gdf for the segment from Cell 3
    print("Error: 'stops_gdf' (for the segment) not found or empty. Please run Cell 3 successfully first.")
elif not os.path.exists(clipped_osm_filepath):
     print(f"Error: Clipped PBF file {clipped_osm_filepath} not found. Please run Cell 3.5 successfully first.")
elif not os.path.exists(alt_paths_filepath):
     print(f"Error: Alternative paths file {alt_paths_filepath} not found. Please run Cell 6 successfully.")
# -----------------------------

else: # Proceed if files exist
    try:
        # 1. Load the alternative paths data (node lists)
        print(f"Loading alternative paths data from {alt_paths_filepath}...")
        with open(alt_paths_filepath, 'rb') as f:
            alternative_paths_data = pickle.load(f)
        print("Alternative paths data loaded.")

        # 2. Load the segment graph AGAIN from the clipped PBF using pyrosm
        print(f"Loading segment graph from clipped PBF: {clipped_osm_filepath} ...")
        start_time_load = time.time()
        osm_reader = OSM(clipped_osm_filepath)
        nodes, edges = osm_reader.get_network(network_type="driving", nodes=True)
        if nodes is None or edges is None or nodes.empty or edges.empty: raise ValueError("Pyrosm failed to extract nodes/edges from clipped file.")
        G = osm_reader.to_graph(nodes, edges, graph_type="networkx")
        del osm_reader, nodes, edges; gc.collect()
        end_time_load = time.time()
        print(f"Loaded segment graph G in {end_time_load - start_time_load:.2f} seconds ({G.number_of_nodes()} nodes, {G.number_of_edges()} edges).")

        # Ensure graph/stops CRS and node attributes are okay
        graph_crs = G.graph.get('crs', "EPSG:4326")
        if stops_gdf.crs != graph_crs: stops_gdf = stops_gdf.to_crs(graph_crs)
        sample_node = list(G.nodes())[0]
        if 'x' not in G.nodes[sample_node] or 'y' not in G.nodes[sample_node]:
            print("Adding 'x'/'y' attributes from node geometry...")
            for node_id, data in G.nodes(data=True):
                 if 'geometry' in data and isinstance(data['geometry'], Point): G.nodes[node_id]['x'], G.nodes[node_id]['y'] = data['geometry'].x, data['geometry'].y

        # 3. Generate Standard Path Geometry (connecting all stops in segment)
        print("Generating geometry for the standard path through segment stops...")
        standard_route_nodes = []
        stop_nodes_viz = ox.nearest_nodes(G, X=stops_gdf.geometry.x.tolist(), Y=stops_gdf.geometry.y.tolist())
        for i in range(len(stop_nodes_viz) - 1):
             start_node, end_node = stop_nodes_viz[i], stop_nodes_viz[i+1]
             if start_node == end_node: continue
             try: path_segment = nx.shortest_path(G, start_node, end_node, weight='length')
             except nx.NetworkXNoPath: continue
             if path_segment:
                 if not standard_route_nodes or path_segment[0] != standard_route_nodes[-1]: standard_route_nodes.extend(path_segment)
                 else: standard_route_nodes.extend(path_segment[1:])
        if standard_route_nodes:
             points = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in standard_route_nodes if node in G.nodes]
             if len(points) >= 2: standard_path_geometry_viz = LineString(points); print(" Standard path geometry created.")

        # 4. Generate Alternative Path B Geometries (using loaded dictionary)
        print("Generating geometries for alternative 'Path B' segments...")
        for start_node, path_data in alternative_paths_data.items():
            if 'B_nodes' in path_data and path_data['B_nodes']:
                alt_nodes = path_data['B_nodes']
                alt_points = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in alt_nodes if node in G.nodes]
                if len(alt_points) >= 2: alternative_geometries_viz.append(LineString(alt_points))
        print(f" Generated {len(alternative_geometries_viz)} alternative path geometries.")

        # 5. Create Folium Map
        print("Creating map...")
        try: map_center_viz = [stops_gdf.geometry.union_all().centroid.y, stops_gdf.geometry.union_all().centroid.x]
        except: map_center_viz = [stops_gdf.iloc[0].geometry.y, stops_gdf.iloc[0].geometry.x]
        map_viz = folium.Map(location=map_center_viz, zoom_start=14)

        # Plot Standard Path (Green)
        if standard_path_geometry_viz:
            std_path_gdf = gpd.GeoDataFrame([{'id': 'Standard (Seg)', 'geometry': standard_path_geometry_viz}], crs=graph_crs)
            folium.GeoJson(
                std_path_gdf.to_json(),
                # *** CHANGED COLOR TO GREEN ***
                style_function=lambda x: {'color': 'green', 'weight': 4, 'opacity': 0.8},
                name='Standard Path connecting Segment Stops'
            ).add_to(map_viz)
            print(" Added standard path (green).")
        else: print(" Standard path geometry not generated for visualization.")

        # Plot Alternative Path B Segments (Red Dashed)
        if alternative_geometries_viz:
            alt_path_gdf = gpd.GeoDataFrame([{'id': f'Alt_{i+1}', 'geometry': geom} for i, geom in enumerate(alternative_geometries_viz)], crs=graph_crs)
            folium.GeoJson(
                alt_path_gdf.to_json(),
                style_function=lambda x: {'color': 'red', 'weight': 5, 'opacity': 0.9, 'dashArray': '5, 5'},
                tooltip=folium.features.GeoJsonTooltip(fields=['id'], aliases=['Path Option:']),
                name='Alternative Path Segments (B)'
            ).add_to(map_viz)
            print(f" Added {len(alternative_geometries_viz)} alternative path segments (red dashed).")
        else: print(" No alternative Path B geometries were generated to plot.")

        # Plot Stops (Blue Markers)
        print(f"Adding {len(stops_gdf)} stop markers...")
        # Define marker cluster if too many stops - optional
        # marker_cluster = folium.plugins.MarkerCluster().add_to(map_viz)
        for idx, row in stops_gdf.iterrows():
             if row.geometry and not row.geometry.is_empty:
                 # Get original sequence index if available
                 seq_idx = stops_gdf.index[idx] + SEGMENT_START_SEQUENCE_IDX if 'SEGMENT_START_SEQUENCE_IDX' in locals() else idx
                 folium.Marker(
                     location=[row.geometry.y, row.geometry.x],
                     popup=f"Stop ID: {row['stop_id']}<br>Seq Index: {seq_idx}",
                     tooltip=f"Stop ID: {row['stop_id']}",
                     icon=folium.Icon(color='blue', icon='bus', prefix='fa')
                 ).add_to(map_viz) # Add directly to map OR to marker_cluster

        # Add click for lat/lon
        folium.LatLngPopup().add_to(map_viz)
        # Add layer control
        folium.LayerControl().add_to(map_viz)

         # --- Add Legend ---
        legend_html = """
        <div style="position: fixed;
                    bottom: 50px; left: 50px; width: 230px; height: 100px; /* Increased width */
                    border:2px solid grey; z-index:9999; font-size:14px;
                    background-color: white; padding: 5px;
                    ">
        &nbsp;<b>Legend</b><br>
        &nbsp;<i class="fa fa-bus" style="color:blue"></i>&nbsp; Bus Stop<br>
        &nbsp;<span style="background-color: green; display: inline-block; width: 20px; height: 3px; vertical-align: middle;"></span>&nbsp; Standard Route Path<br>
        &nbsp;<span style="border-bottom: 5px dashed red; display: inline-block; width: 20px; vertical-align: middle;"></span>&nbsp; Alternative Route Segment<br>
        </div>
        """
        legend = branca.element.Element(legend_html)
        map_viz.get_root().html.add_child(legend)
        print(" Added map legend.")
        # ------------------

        # Save (optional) and display the map
        map_filename_viz = f"/content/trip_{TARGET_TRIP_ID}_segment_alternatives_map.html"
        map_viz.save(map_filename_viz)
        print(f"\nSaved visualization map to {map_filename_viz}.")
        print("Map created. Displaying...")
        display(map_viz) # Display the map

    except FileNotFoundError as e_fnf: print(f"Error: Required file not found - {e_fnf}")
    except Exception as e_viz: print(f"An error occurred during visualization: {e_viz}"); traceback.print_exc()
    finally:
        # Cleanup loaded graph
        if 'G' in locals() and G is not None: del G; gc.collect(); print("Cleaned up graph object.")
        else: print("Graph object G was not created or already deleted.")

# else block from prerequisite check
# else:
#    print("Skipping visualization due to missing prerequisites.")

print("Visualization cell complete.")


--- Loading Data, Generating Geometries, and Visualizing ---
Loading alternative paths data from /content/alt_paths_362_13_36.pkl...
Alternative paths data loaded.
Loading segment graph from clipped PBF: /content/delhi_segment_clipped.osm.pbf ...
Loaded segment graph G in 35.26 seconds (171857 nodes, 396956 edges).
Generating geometry for the standard path through segment stops...
 Standard path geometry created.
Generating geometries for alternative 'Path B' segments...
 Generated 3 alternative path geometries.
Creating map...
 Added standard path (green).
 Added 3 alternative path segments (red dashed).
Adding 94 stop markers...
 Added map legend.

Saved visualization map to /content/trip_362_13_36_segment_alternatives_map.html.
Map created. Displaying...


Cleaned up graph object.
Visualization cell complete.


In [ ]:
# Cell 7: Load Graph (from Clipped PBF), Alternatives (from PKL), and Stops for PSO

import osmnx as ox
import networkx as nx
import geopandas as gpd
import pickle
import os
import gc
import time
import traceback
try:
    from pyrosm import OSM # Requires pyrosm installation
except ImportError: print("Error: pyrosm library not found."); raise

print("\n--- Loading Inputs for PSO ---")

# --- Define Filepaths for the FULL Route ---
# Ensure these match the TARGET_TRIP_ID used for the successful full run
TARGET_TRIP_ID_PSO = '362_13_36' # The 94-stop trip ID

# Path to the CLIPPED PBF file created by Cell 3.5 for the FULL trip
clipped_osm_filepath_full = f'/content/delhi_segment_clipped.osm.pbf' # NOTE: Filename convention assumed, adjust if Cell 3.5 saved differently!

# Path to the alternatives data saved by Cell 6 (for the full trip graph)
alt_paths_filepath_full = f'/content/alt_paths_{TARGET_TRIP_ID_PSO}.pkl' # Assuming Cell 6 saved with this name for the full trip
# ------------------------------------------

# Initialize variables
G_pso = None
alternative_paths_data_pso = None
stops_gdf_pso = None # Will reference stops_gdf from Cell 3

# --- Check for Stops GeoDataFrame (Prerequisite from Cell 3) ---
print("Checking for stops_gdf (full route)...")
if 'stops_gdf' not in locals() or stops_gdf.empty:
    print("Error: 'stops_gdf' (for the full route) not found or empty.")
    print("Please ensure Cell 3 (processing ALL stops) was run successfully before this cell.")
    # Optionally add logic here to try and re-run cell 3 if needed, or raise error
    raise NameError("stops_gdf not available for PSO.")
else:
    # Assuming stops_gdf from Cell 3 is the one we need for the full route
    stops_gdf_pso = stops_gdf
    print(f"Found stops_gdf with {len(stops_gdf_pso)} stops.")
    # You might want to check if len(stops_gdf_pso) matches expected (e.g., 94)

# --- Load Alternative Paths Data (Prerequisite from Cell 6) ---
print(f"\nAttempting to load alternative paths data from: {alt_paths_filepath_full}")
if not os.path.exists(alt_paths_filepath_full):
    print(f"Error: Alternative paths file not found at {alt_paths_filepath_full}.")
    print("Please ensure Cell 6 was run successfully for this trip ID.")
    alternative_paths_data_pso = None # Indicate failure
else:
    try:
        with open(alt_paths_filepath_full, 'rb') as f:
            alternative_paths_data_pso = pickle.load(f)
        print(f"Successfully loaded alternative paths data ({len(alternative_paths_data_pso)} segments with alternatives).")
    except Exception as e_pickle:
        print(f"An error occurred loading alternative paths data: {e_pickle}")
        alternative_paths_data_pso = None # Ensure it's None on error

# --- Load Graph from Clipped PBF using Pyrosm (Avoids load_graphml error) ---
# This section regenerates G based on the clipped PBF for the full route area
if alternative_paths_data_pso is not None: # Only proceed if alternatives loaded okay
    print(f"\nAttempting to load graph from clipped PBF: {clipped_osm_filepath_full}")
    if not os.path.exists(clipped_osm_filepath_full):
        print(f"Error: Clipped PBF file not found at {clipped_osm_filepath_full}")
        print("Please ensure Cell 3.5 was run successfully for the full route.")
        G_pso = None # Indicate failure
    else:
        try:
            print(f"Initializing pyrosm reader for clipped file: {clipped_osm_filepath_full} ...")
            start_time_load = time.time()
            osm_reader = OSM(clipped_osm_filepath_full)
            nodes, edges = osm_reader.get_network(network_type="driving", nodes=True)
            if nodes is None or edges is None or nodes.empty or edges.empty: raise ValueError("Pyrosm failed to extract nodes/edges from clipped file.")
            print("Converting pyrosm network to NetworkX graph...")
            G_pso = osm_reader.to_graph(nodes, edges, graph_type="networkx")
            del osm_reader, nodes, edges; gc.collect()
            end_time_load = time.time()
            print(f"Successfully loaded graph G_pso in {end_time_load - start_time_load:.2f} seconds.")
            print(f"Graph G_pso has {G_pso.number_of_nodes()} nodes and {G_pso.number_of_edges()} edges.")

            # Add x/y attributes if missing (needed for ox.nearest_nodes later maybe)
            sample_node = list(G_pso.nodes())[0]
            if 'x' not in G_pso.nodes[sample_node] or 'y' not in G_pso.nodes[sample_node]:
                 print("Adding 'x'/'y' attributes from node geometry...")
                 for node_id, data in G_pso.nodes(data=True):
                      if 'geometry' in data and isinstance(data['geometry'], Point): G_pso.nodes[node_id]['x'], G_pso.nodes[node_id]['y'] = data['geometry'].x, data['geometry'].y

        except FileNotFoundError: print(f"Error: Clipped PBF file not found at '{clipped_osm_filepath_full}'.")
        except ImportError: print("Error: Could not import pyrosm.")
        except Exception as e_load: print(f"An error occurred loading graph from PBF: {e_load}"); traceback.print_exc(); G_pso = None

# --- Final Check ---
if G_pso is not None and alternative_paths_data_pso is not None and stops_gdf_pso is not None and not stops_gdf_pso.empty:
    print("\n--- PSO Inputs Ready ---")
    print(f" Graph G_pso: {G_pso.number_of_nodes()} nodes, {G_pso.number_of_edges()} edges")
    print(f" Stops stops_gdf_pso: {len(stops_gdf_pso)} stops loaded")
    print(f" Alternatives alternative_paths_data_pso: Data loaded for {len(alternative_paths_data_pso)} decision points")
    # Graph G_pso will remain in memory for the fitness function to use
else:
    print("\n--- Error: Not all PSO inputs could be loaded successfully. Please check previous steps. ---")
    # Clean up graph if partially loaded
    if G_pso is not None: del G_pso; G_pso = None; gc.collect(); print("Cleaned up partially loaded graph.")

print("Cell 7/?: PSO Setup attempt complete.")
# Variables available for next steps: G_pso, alternative_paths_data_pso, stops_gdf_pso (if all loaded)


--- Loading Inputs for PSO ---
Checking for stops_gdf (full route)...
Found stops_gdf with 94 stops.

Attempting to load alternative paths data from: /content/alt_paths_362_13_36.pkl
Successfully loaded alternative paths data (3 segments with alternatives).

Attempting to load graph from clipped PBF: /content/delhi_segment_clipped.osm.pbf
Initializing pyrosm reader for clipped file: /content/delhi_segment_clipped.osm.pbf ...
Converting pyrosm network to NetworkX graph...
Successfully loaded graph G_pso in 33.23 seconds.
Graph G_pso has 171857 nodes and 396956 edges.

--- PSO Inputs Ready ---
 Graph G_pso: 171857 nodes, 396956 edges
 Stops stops_gdf_pso: 94 stops loaded
 Alternatives alternative_paths_data_pso: Data loaded for 3 decision points
Cell 7/?: PSO Setup attempt complete.


In [ ]:
# Cell 8: PSO Particle Definition
import numpy as np

class Particle:
    """
    Represents a single particle in the Particle Swarm Optimization.
    """
    def __init__(self, num_dimensions):
        """
        Initializes a particle.

        Args:
            num_dimensions (int): The number of dimensions in the search space
                                  (should be 3 for our 3 decision points).
        """
        self.num_dimensions = num_dimensions

        # Position: Represents the particle's current solution (tendency for Path B).
        # Initialize each element randomly between 0.0 and 1.0.
        self.position = np.random.rand(num_dimensions)

        # Velocity: Represents the particle's movement direction and speed.
        # Initialize with small random values (e.g., between 0.0 and 0.1).
        self.velocity = np.random.rand(num_dimensions) * 0.1

        # Personal Best Position: The best position found by this particle so far.
        # Initially set to its starting position.
        self.best_position = self.position.copy()

        # Fitness: The score of the particle's current position (lower is better).
        # Will be calculated by the fitness function. Initialized to infinity.
        self.fitness = float('inf')

        # Personal Best Fitness: The best score achieved by this particle so far.
        # Initialized to infinity.
        self.best_fitness = float('inf')

    def __str__(self):
        # Optional: For printing particle state easily
        return (f"Pos: {np.round(self.position, 2)}, "
                f"Vel: {np.round(self.velocity, 2)}, "
                f"BestPos: {np.round(self.best_position, 2)}, "
                f"Fit: {self.fitness:.2f}, BestFit: {self.best_fitness:.2f}")

print("Particle class defined.")

# --- Example of creating one particle for our problem ---
num_decision_points = 3
p1 = Particle(num_dimensions=num_decision_points)
print("Example Particle Initial State:")
print(p1)
# --------------------------------------------------------

Particle class defined.
Example Particle Initial State:
Pos: [0.4  0.41 0.18], Vel: [0.09 0.04 0.  ], BestPos: [0.4  0.41 0.18], Fit: inf, BestFit: inf


In [ ]:
# Cell 9: Decoding Function

def decode_choices(particle_position):
    """
    Decodes a particle's continuous position vector (0.0 to 1.0)
    into discrete path choices ('A' or 'B') using a threshold.

    Args:
        particle_position (np.array): The particle's position vector
                                      (e.g., a 3-element numpy array).

    Returns:
        list: A list of strings ('A' or 'B') representing the choices
              for each decision point/segment.
              Example: ['B', 'A', 'B']
    """
    choices = []
    threshold = 0.5
    # Check each dimension against the threshold
    for i in range(len(particle_position)):
        if particle_position[i] >= threshold:
            choices.append('B') # Value >= threshold means choose Alternative Path B
        else:
            choices.append('A') # Value < threshold means choose Standard Path A
    return choices

print("decode_choices function defined.")

# --- Example usage: ---
# example_position = np.array([0.7, 0.2, 0.9])
# example_choices = decode_choices(example_position)
# print(f"\nDecoding example position {example_position}:")
# print(f" -> Choices: {example_choices}") # Output should be ['B', 'A', 'B']
# --------------------

decode_choices function defined.


In [ ]:
# Cell: Visualize Path Connecting Consecutive Stops
# Purpose: Load the graph (G_pso), find nodes for all stops in sequence,
#          calculate the shortest path between *each consecutive pair* of stop nodes,
#          stitch these paths together, and visualize the result.
# Assumes Cell 7 (PSO Setup) has been run successfully.

import osmnx as ox
import networkx as nx
import geopandas as gpd
import folium
import time
import gc
import os
import traceback
from shapely.geometry import Point, LineString # For node attribute check
import branca.element # For legend

print("\n--- Visualizing Path Connecting Consecutive Stops ---")

# --- Configuration ---
# Ensure this matches the TARGET_TRIP_ID used in Cell 7
TARGET_TRIP_ID_VIZ_CONS = '362_13_36'
# --------------------

# Initialize variables
G_viz_cons = None
route_nodes_cons = []
map_viz_cons = None

# --- Prerequisites Check ---
if ('G_pso' not in locals() or G_pso is None or
    'stops_gdf_pso' not in locals() or stops_gdf_pso.empty):
    print("Error: Required data (G_pso, stops_gdf_pso) not found.")
    print("Please ensure Cell 7 (PSO Setup) ran successfully first.")
    raise NameError("Required data G_pso or stops_gdf_pso not available.")
else:
    # Use the already loaded graph and stops
    G_viz_cons = G_pso
    stops_gdf_viz_cons = stops_gdf_pso.copy() # Use a copy
    print(f"Using loaded G_pso ({G_viz_cons.number_of_nodes()} nodes) and stops_gdf_pso ({len(stops_gdf_viz_cons)} stops).")

    try:
        # === Part 1: Find Nearest Nodes for ALL Stops ===
        print("Finding nearest nodes for all stops in sequence...")
        start_time_nodes = time.time()
        # Ensure CRS match if graph has one
        graph_crs_viz = G_viz_cons.graph.get('crs', "EPSG:4326")
        if stops_gdf_viz_cons.crs != graph_crs_viz:
            stops_gdf_viz_cons = stops_gdf_viz_cons.to_crs(graph_crs_viz)

        # Get nearest node for each stop geometry
        stop_nodes_viz = ox.nearest_nodes(G_viz_cons,
                                          X=stops_gdf_viz_cons.geometry.x.tolist(),
                                          Y=stops_gdf_viz_cons.geometry.y.tolist())
        stops_gdf_viz_cons['nearest_node'] = stop_nodes_viz
        end_time_nodes = time.time()
        print(f"Found nearest nodes for {len(stop_nodes_viz)} stops in {end_time_nodes - start_time_nodes:.2f} seconds.")

        # === Part 2: Calculate Path by Connecting Consecutive Stop Nodes ===
        print("\nCalculating path by connecting consecutive stop nodes...")
        start_time_path = time.time()
        route_nodes_cons = []
        valid_segments = 0
        # Loop through pairs of consecutive stops
        for i in range(len(stops_gdf_viz_cons) - 1):
            orig_node = stops_gdf_viz_cons.iloc[i]['nearest_node']
            dest_node = stops_gdf_viz_cons.iloc[i+1]['nearest_node']

            # Skip if origin and destination are the same node
            if orig_node == dest_node:
                # print(f"  Skipping segment {i}: Origin node == Destination node ({orig_node})")
                # Add the node once if it's the start or different from the last added node
                if not route_nodes_cons or orig_node != route_nodes_cons[-1]:
                     route_nodes_cons.append(orig_node)
                continue

            try:
                # Find the shortest path between this pair of stop nodes
                path_segment = nx.shortest_path(G_viz_cons, source=orig_node, target=dest_node, weight='length')

                # Append the path segment nodes, avoiding duplicates at the joins
                if not route_nodes_cons:
                    # First segment, add all nodes
                    route_nodes_cons.extend(path_segment)
                else:
                    # Subsequent segments, check for connection and avoid duplicate node
                    if path_segment and path_segment[0] == route_nodes_cons[-1]:
                         route_nodes_cons.extend(path_segment[1:]) # Add all except the first node
                    elif path_segment: # If disconnected, just append (might look weird but better than skipping)
                         print(f"  Warning: Segment {i} seems disconnected (start node {path_segment[0]} != previous end {route_nodes_cons[-1]})? Appending anyway.")
                         route_nodes_cons.extend(path_segment)
                    # If path_segment is empty (shouldn't happen if nx.shortest_path succeeded), do nothing

                valid_segments += 1
            except nx.NetworkXNoPath:
                print(f"  Warning: No path found between stop {i} (node {orig_node}) and stop {i+1} (node {dest_node}). Skipping segment.")
            except Exception as e_path:
                print(f"  Warning: Error calculating path for segment {i} ({orig_node} -> {dest_node}): {e_path}")

        end_time_path = time.time()
        print(f"Finished calculating path segments in {end_time_path - start_time_path:.2f} seconds.")
        print(f"Successfully calculated {valid_segments}/{len(stops_gdf_viz_cons) - 1} segments.")
        print(f"Total nodes in stitched path: {len(route_nodes_cons)}")

        # === Part 3: Create Visualization ===
        print("\nCreating map...")
        # Use the GeoDataFrame for stops which might have been reprojected
        try:
             map_center_viz = [stops_gdf_viz_cons.geometry.unary_union().centroid.y, stops_gdf_viz_cons.geometry.unary_union().centroid.x]
        except Exception as center_e:
             print(f"Warning: Could not calculate map center: {center_e}. Using first stop.")
             map_center_viz = [stops_gdf_viz_cons.iloc[0].geometry.y, stops_gdf_viz_cons.iloc[0].geometry.x] # Fallback

        map_viz_cons = folium.Map(location=map_center_viz, zoom_start=12) # Adjust zoom as needed

        # Plot the Stitched Path (e.g., Green)
        if route_nodes_cons and len(route_nodes_cons) >= 2:
            # Extract coordinates for the path
            points_viz = []
            for node in route_nodes_cons:
                 if node in G_viz_cons.nodes:
                      points_viz.append((G_viz_cons.nodes[node]['y'], G_viz_cons.nodes[node]['x']))
                 else:
                      print(f"Warning: Node {node} from path not found in graph G_viz_cons during coordinate extraction.")

            if len(points_viz) >= 2:
                folium.PolyLine(
                    points_viz,
                    color='green',
                    weight=5,
                    opacity=0.8,
                    tooltip='Path Connecting Stops',
                    name='Consecutive Stop Path'
                ).add_to(map_viz_cons)
                print("Added stitched path connecting stops (green) to map.")
            else:
                print("Warning: Could not extract sufficient coordinates for path polyline.")
        else:
            print("Warning: No valid path nodes found to plot.")

        # Plot Stops (Red Markers)
        print(f"Adding {len(stops_gdf_viz_cons)} stop markers...")
        for idx, row in stops_gdf_viz_cons.iterrows():
              if row.geometry and not row.geometry.is_empty:
                  folium.Marker(
                      location=[row.geometry.y, row.geometry.x],
                      popup=f"Stop ID: {row['stop_id']}<br>Seq Index: {idx}", # Use index as sequence proxy
                      tooltip=f"Stop ID: {row['stop_id']}",
                      icon=folium.Icon(color='red', icon='bus', prefix='fa')
                  ).add_to(map_viz_cons)

        # Add Legend
        legend_html = """
             <div style="position: fixed; bottom: 50px; left: 50px; width: 220px; height: 70px;
                         border:2px solid grey; z-index:9999; font-size:14px;
                         background-color: white; padding: 5px;">
             &nbsp;<b>Legend</b><br>
             &nbsp;<i class="fa fa-bus" style="color:red"></i>&nbsp; Bus Stop<br>
             &nbsp;<span style="background-color: green; display: inline-block; width: 20px; height: 4px; vertical-align: middle;"></span>&nbsp; Shortest Route Found<br>
             </div> """
        legend = branca.element.Element(legend_html)
        map_viz_cons.get_root().html.add_child(legend)

        # Add Layer Control
        folium.LayerControl().add_to(map_viz_cons)

        # Save and display
        map_filename_cons = f"/content/consecutive_stops_path_map_{TARGET_TRIP_ID_VIZ_CONS}.html"
        map_viz_cons.save(map_filename_cons)
        print(f"\nSaved consecutive stops path map to {map_filename_cons}.")
        print("Displaying map...")
        display(map_viz_cons)

    except Exception as e_viz:
        print(f"An error occurred during visualization: {e_viz}")
        traceback.print_exc()
    finally:
        # Clean up graph object if it was copied locally (it wasn't here, using G_pso)
        # del G_viz_cons # Don't delete G_pso here
        if 'stops_gdf_viz_cons' in locals(): del stops_gdf_viz_cons
        gc.collect()
        print("Cleaned up visualization variables.")

# else block from prerequisite check finishes here
# else:
#    print("Skipping consecutive stop path visualization: Prerequisites not met.")

print("\nCell: Consecutive Stop Path Visualization complete.")


--- Visualizing Path Connecting Consecutive Stops ---
Using loaded G_pso (171857 nodes) and stops_gdf_pso (94 stops).
Finding nearest nodes for all stops in sequence...
Found nearest nodes for 94 stops in 1.85 seconds.

Calculating path by connecting consecutive stop nodes...
Finished calculating path segments in 0.25 seconds.
Successfully calculated 92/93 segments.
Total nodes in stitched path: 1634

Creating map...
Added stitched path connecting stops (green) to map.
Adding 94 stop markers...


<ipython-input-17-c4ca107ac3fe>:109: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  map_center_viz = [stops_gdf_viz_cons.geometry.unary_union().centroid.y, stops_gdf_viz_cons.geometry.unary_union().centroid.x]



Saved consecutive stops path map to /content/consecutive_stops_path_map_362_13_36.html.
Displaying map...


Cleaned up visualization variables.

Cell: Consecutive Stop Path Visualization complete.


In [ ]:
# Cell 10: API Helper Functions (TomTom & Weather Only)

import requests
import json
import time
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, LineString
# Removed datetime, timezone, gtfs_realtime_pb2 as they were for the live bus location function

# Weather API libraries (ensure installed)
try:
    import openmeteo_requests
    import requests_cache
    from retry_requests import retry
except ImportError:
    print("WARNING: Weather libraries not found. Weather functions will not work.")
    openmeteo_requests = None

# --- API Keys ---
TOMTOM_API_KEY = "TBo0EKP7kmbDlw9UG3j8QJQTGi3nvFTk" # User's preferred key
# Removed DELHI_API_KEY and DELHI_API_URL
# ----------------

# --- TomTom Traffic Flow ---
def get_tomtom_traffic_flow(lat, lon, api_key):
    """Fetches real-time traffic flow data (speed) from TomTom API."""
    url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json?key={api_key}&point={lat},{lon}"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json();
        if data and 'flowSegmentData' in data: return data['flowSegmentData']
        else: return None
    except requests.exceptions.RequestException as e: print(f"Warning: TomTom Flow API request failed: {e}"); return None
    except json.JSONDecodeError: print("Warning: TomTom Flow API returned invalid JSON."); return None

# --- TomTom Incidents ---
def get_tomtom_traffic_incidents(bbox_str, api_key):
    """ Fetches real-time traffic incidents from TomTom API for a bounding box. """
    category_filter = "1,6,8"; fields_param = "{incidents{type,geometry{type,coordinates},properties{iconCategory}}}"
    url = (f"https://api.tomtom.com/traffic/services/5/incidentDetails?"
           f"key={api_key}&bbox={bbox_str}&fields={fields_param}"
           f"&language=en-GB&timeValidityFilter=present&categoryFilter={category_filter}")
    try:
        response = requests.get(url, timeout=15); response.raise_for_status(); data = response.json()
        incident_details = []
        if data and 'incidents' in data:
            for incident in data['incidents']:
                 props = incident.get('properties', {}); category = props.get('iconCategory'); geom_data = incident.get('geometry');
                 if geom_data and category is not None:
                      geom_type = geom_data.get('type'); coords = geom_data.get('coordinates'); geometry = None
                      try:
                          if geom_type == 'Point' and coords: geometry = Point(coords)
                          elif geom_type == 'LineString' and coords: geometry = LineString(coords)
                      except Exception as geom_e: print(f"Warning: Could not parse incident geometry: {geom_e}")
                      if geometry: incident_details.append({'category': category, 'geometry': geometry})
            return incident_details
        else: return []
    except requests.exceptions.RequestException as e: print(f"Warning: TomTom Incidents API request failed: {e}"); return None
    except json.JSONDecodeError: print("Warning: TomTom Incidents API returned invalid JSON."); return None

# --- Open-Meteo Weather ---
def get_current_weather(lat, lon):
    """Fetches current weather using Open-Meteo, including weather code and precipitation."""
    if openmeteo_requests is None: return None
    try:
        cache_session = requests_cache.CachedSession('.cache', expire_after=900); retry_session = retry(cache_session, retries=3, backoff_factor=0.2)
        openmeteo = openmeteo_requests.Client(session=retry_session); url = "https://api.open-meteo.com/v1/forecast"
        params = { "latitude": lat, "longitude": lon, "current": "weather_code,precipitation", "timezone": "auto" }
        responses = openmeteo.weather_api(url, params=params); response = responses[0]; current = response.Current()
        weather_data = {"weather_code": current.Variables(0).Value(), "precipitation": current.Variables(1).Value()}
        return weather_data
    except Exception as e: print(f"Warning: Open-Meteo API Error: {e}"); return None

# --- Delhi Live Bus Location Function REMOVED ---

print("Cell 8: API helper functions defined (TomTom Traffic, TomTom Incidents, Weather).")

Cell 8: API helper functions defined (TomTom Traffic, TomTom Incidents, Weather).


In [ ]:
# Cell 11: Real-Time Fitness Function Definition (Fixing Deprecation)

import numpy as np
import networkx as nx
import geopandas as gpd
from shapely.geometry import Point, LineString # Make sure LineString is imported
import time
import warnings
import traceback # For detailed errors

# --- Constants/Assumptions ---
DEFAULT_SPEED_KMH = 30.0
MIN_SPEED_KMH = 5.0
DEFAULT_SPEED_MPS = DEFAULT_SPEED_KMH * 1000 / 3600

# Weather Factors
FOG_CODES = range(40, 50); RAIN_CODES = range(50, 70); SHOWER_CODES = range(80, 83)
RAIN_PRECIP_THRESHOLD_MM = 1.0
WEATHER_FACTOR_FOG = 1.30; WEATHER_FACTOR_RAIN = 1.05; WEATHER_FACTOR_DEFAULT = 1.0

# Incident Penalties & Check Distance
INCIDENT_JAM_PENALTY_SECONDS = 300
INCIDENT_CLOSURE_PENALTY_SECONDS = float('inf')
INCIDENT_PROXIMITY_BUFFER_DEG = 0.0002

# API Call settings
API_PAUSE_SECONDS = 0.05
# -----------------------------

# Make sure loaded data from Cell 7 exists: G_pso, alternative_paths_data_pso, stops_gdf_pso
# Make sure decode_choices and API helpers (Cell 8) are defined

# --- Cache for standard path segments ---
memo_std_path_nodes = {}
def get_standard_path_nodes(G, start_node, end_node):
    if start_node == end_node: return [start_node] if start_node in G else []
    segment_key = (start_node, end_node)
    if segment_key in memo_std_path_nodes: return memo_std_path_nodes[segment_key]
    try:
        nodes = nx.shortest_path(G, start_node, end_node, weight='length')
        memo_std_path_nodes[segment_key] = nodes; return nodes
    except (nx.NetworkXNoPath, nx.NodeNotFound): print(f"Warning: No standard path {start_node}->{end_node}"); return []
    except Exception as e: print(f"Warning: Error standard path {start_node}->{end_node}: {e}"); return []


# --- Real-Time Fitness Function ---
def calculate_realtime_fitness(particle_position, G, alternatives_data, stops_gdf):
    print(">>> Entering calculate_realtime_fitness function <<<")
    global TOMTOM_API_KEY
    if G is None or not alternatives_data or stops_gdf.empty or len(stops_gdf) < 2: return float('inf')

    # --- Pre-computation ---
    start_time_fitness = time.time()
    current_incidents = None
    try: # Fetch Incidents
        min_lon, min_lat, max_lon, max_lat = stops_gdf.total_bounds
        buffer = 0.01; bbox_str = f"{min_lon-buffer},{min_lat-buffer},{max_lon+buffer},{max_lat+buffer}"
        current_incidents = get_tomtom_traffic_incidents(bbox_str, TOMTOM_API_KEY)
        if current_incidents is None: print("Warning: Failed getting incidents.")
    except Exception as e: print(f"Error fetching incidents: {e}"); current_incidents = None

    current_weather = None
    try: # Get Weather
        # *** USE union_all() instead of unary_union ***
        center_lat, center_lon = stops_gdf.geometry.union_all().centroid.y, stops_gdf.geometry.union_all().centroid.x
        current_weather = get_current_weather(center_lat, center_lon)
        if current_weather is None: print("Warning: Failed getting weather.")
    except Exception as e: print(f"Error getting weather: {e}"); current_weather = None # Keep None on error

    # Determine Weather Factor
    weather_factor = WEATHER_FACTOR_DEFAULT
    if current_weather:
        code = current_weather.get('weather_code', -1); precip = current_weather.get('precipitation', 0)
        if code in FOG_CODES: weather_factor = WEATHER_FACTOR_FOG
        elif code in RAIN_CODES or code in SHOWER_CODES or precip >= RAIN_PRECIP_THRESHOLD_MM: weather_factor = WEATHER_FACTOR_RAIN

    # Decode Choices & Identify Key Nodes
    choices = decode_choices(particle_position)
    dp_start_nodes = sorted(list(alternatives_data.keys()))
    if len(dp_start_nodes) != 3 or len(choices) != 3: return float('inf')
    dp1_node, dp2_node, dp3_node = dp_start_nodes
    rp1_node = alternatives_data[dp1_node]['end_node']; rp2_node = alternatives_data[dp2_node]['end_node']; rp3_node = alternatives_data[dp3_node]['end_node']
    try:
        stop_nodes_list = ox.nearest_nodes(G, X=stops_gdf.geometry.x.tolist(), Y=stops_gdf.geometry.y.tolist())
        segment_start_node = stop_nodes_list[0]; segment_end_node = stop_nodes_list[-1]
    except Exception as e: print(f"Error getting nearest nodes in fitness: {e}"); return float('inf')

    # Select Path Segments (Node Lists) based on Choices
    full_route_segments_nodes = []
    try:
        full_route_segments_nodes.append(get_standard_path_nodes(G, segment_start_node, dp1_node))
        full_route_segments_nodes.append(alternatives_data[dp1_node][choices[0] + '_nodes'])
        full_route_segments_nodes.append(get_standard_path_nodes(G, rp1_node, dp2_node))
        full_route_segments_nodes.append(alternatives_data[dp2_node][choices[1] + '_nodes'])
        full_route_segments_nodes.append(get_standard_path_nodes(G, rp2_node, dp3_node))
        full_route_segments_nodes.append(alternatives_data[dp3_node][choices[2] + '_nodes'])
        full_route_segments_nodes.append(get_standard_path_nodes(G, rp3_node, segment_end_node))
        if any(not segment for segment in full_route_segments_nodes): raise ValueError("Could not construct all path segments.")
    except Exception as e_stitch: print(f"Error during path segment selection: {e_stitch}"); return float('inf')

    # Estimate Travel Time for the Stitched Path
    total_estimated_time_seconds = 0.0
    api_call_count = 0

    print(f"  Fitness eval for choices {choices} (Weather factor: {weather_factor:.2f}): Estimating time...")

    # --- Iterate through the 7 logical segments of the path ---
    for segment_idx, segment_nodes in enumerate(full_route_segments_nodes):
        if len(segment_nodes) < 2: continue
        segment_length_m, segment_time_sec, segment_geom = 0.0, 0.0, None
        try: # Calculate Segment Length and Geometry
            segment_length_m = nx.path_weight(G, segment_nodes, weight='length')
            points = [(G.nodes[node]['x'], G.nodes[node]['y']) for node in segment_nodes if node in G.nodes]
            if len(points) >= 2: segment_geom = LineString(points)
        except Exception as e: print(f"Warning: length/geom failed seg {segment_idx}: {e}"); total_estimated_time_seconds=float('inf'); break
        if segment_length_m <= 0: continue

        # Estimate Segment Speed using TomTom Flow API
        current_speed_kmh = DEFAULT_SPEED_KMH
        if segment_geom:
            try:
                mid_point = segment_geom.interpolate(0.5, normalized=True); mid_lat, mid_lon = mid_point.y, mid_point.x
                flow_info = get_tomtom_traffic_flow(mid_lat, mid_lon, TOMTOM_API_KEY)
                api_call_count += 1; time.sleep(API_PAUSE_SECONDS)
                if flow_info and flow_info.get('currentSpeed') is not None: current_speed_kmh = max(flow_info['currentSpeed'], MIN_SPEED_KMH)
            except Exception as api_e: print(f"Warning: TomTom API call failed segment {segment_idx}: {api_e}")

        current_speed_mps = current_speed_kmh * 1000 / 3600
        segment_base_time = segment_length_m / current_speed_mps if current_speed_mps > 0 else float('inf')

        # Check for Incidents Affecting This Segment
        incident_penalty = 0
        if current_incidents and segment_geom:
             segment_buffer = segment_geom.buffer(INCIDENT_PROXIMITY_BUFFER_DEG)
             for incident in current_incidents:
                  try:
                      if incident['geometry'].intersects(segment_buffer):
                           if incident['category'] == 8: incident_penalty = INCIDENT_CLOSURE_PENALTY_SECONDS; break
                           elif incident['category'] in [1, 6]: incident_penalty = max(incident_penalty, INCIDENT_JAM_PENALTY_SECONDS)
                  except Exception as e_inc: print(f"Warning: Incident check error: {e_inc}")

        # Combine base time, incident penalty, and weather factor
        segment_time_sec = segment_base_time + incident_penalty
        segment_time_sec *= weather_factor
        total_estimated_time_seconds += segment_time_sec
        if total_estimated_time_seconds == float('inf'): break
    # --- End Segment Loop ---

    end_time_fitness = time.time()
    print(f"      Fitness eval time: {end_time_fitness - start_time_fitness:.3f}s (API calls: {api_call_count})")

    # Return Fitness
    return total_estimated_time_seconds

print("calculate_realtime_fitness function defined (Weather, Incidents, Traffic Integrated, Deprecation Fixed).")




calculate_realtime_fitness function defined (Weather, Incidents, Traffic Integrated, Deprecation Fixed).


In [ ]:
# --- Example Usage ---
# (Leave example commented out unless specifically testing this function in isolation)
print("DEBUG: About to run fitness function example...")
try:
     if G_pso is not None:
         seg_start_node = ox.nearest_nodes(G_pso, Y=stops_gdf_pso.iloc[0].geometry.y, X=stops_gdf_pso.iloc[0].geometry.x)
         seg_end_node = ox.nearest_nodes(G_pso, Y=stops_gdf_pso.iloc[-1].geometry.y, X=stops_gdf_pso.iloc[-1].geometry.x)
         dummy_particle = Particle(num_dimensions=3)
         dummy_particle.position = np.array([0.1, 0.6, 0.4]) # Example: Choose A, B, A
         fitness_value = calculate_realtime_fitness(dummy_particle.position, G_pso, alternative_paths_data_pso, stops_gdf_pso)
         print(f"\nExample real-time fitness calculation for position {dummy_particle.position}:")
         print(f" -> Decoded Choices: {decode_choices(dummy_particle.position)}")
         print(f" -> Total Est. Time (Fitness): {fitness_value:.2f} seconds")
     else: print("\nCannot run example: G_pso not loaded.")
except NameError as ne: print(f"\nCannot run example: Required variable missing ({ne})")
except Exception as ex: print(f"\nError in example: {ex}")

DEBUG: About to run fitness function example...

Cannot run example: G_pso not loaded.


In [ ]:
# Cell 10: PSO Main Loop Implementation (Using Real-Time Fitness & History)

import numpy as np
import time
import osmnx as ox # Needed for nearest_nodes if called here
import networkx as nx # Needed for error checking maybe
import gc # For cleanup

# Make sure Particle class and decode_choices function are defined
# Make sure calculate_realtime_fitness function is defined (Cell 9)
# Make sure G_pso, alternative_paths_data_pso, stops_gdf_pso are loaded (Cell 7)

print("\n--- Starting PSO Optimization (Based on Real-Time Estimate) ---")

# --- PSO Parameters ---
# !!! Adjust these for a proper run vs. quick test !!!
num_particles = 5      # Swarm size (e.g., 20-50)
num_dimensions = 3       # Number of decision points (should be 3)
max_iterations = 3    # Number of iterations to run (e.g., 50-100)
# --- Coefficients ---
inertia_weight = 0.7     # w
cognitive_coeff = 1.5    # c1
social_coeff = 1.5       # c2
position_min = 0.0
position_max = 1.0
# --------------------

# Initialize global best tracking
global_best_position = None
global_best_fitness = float('inf')
swarm = []
fitness_history = [] # <<< NEW: List to store best fitness per iteration

# --- Initialization ---
try:
    print(f"Initializing {num_particles} particles...")
    # Check if necessary data is loaded
    if 'G_pso' not in locals() or 'alternative_paths_data_pso' not in locals() or 'stops_gdf_pso' not in locals() \
       or G_pso is None or alternative_paths_data_pso is None or stops_gdf_pso.empty:
        raise NameError("PSO input data (G_pso, alternatives, stops) missing or invalid. Run Cell 7.")

    swarm = [Particle(num_dimensions) for _ in range(num_particles)] # Create particles

    # Get start/end nodes
    print("Finding start/end nodes for segment...")
    if G_pso.graph.get('crs') and stops_gdf_pso.crs != G_pso.graph['crs']:
        stops_gdf_pso = stops_gdf_pso.to_crs(G_pso.graph['crs'])
    segment_stop_nodes = ox.nearest_nodes(G_pso, X=stops_gdf_pso.geometry.x.tolist(), Y=stops_gdf_pso.geometry.y.tolist())
    segment_start_node = segment_stop_nodes[0]
    segment_end_node = segment_stop_nodes[-1]
    print(f"Segment Start Node: {segment_start_node}, Segment End Node: {segment_end_node}")

    print("Calculating initial fitness for swarm (using real-time)...")
    initial_eval_start_time = time.time()
    for i, particle in enumerate(swarm):
        print(f"  Evaluating initial particle {i+1}/{num_particles}...")
        particle.fitness = calculate_realtime_fitness(particle.position, G_pso, alternative_paths_data_pso, stops_gdf_pso)
        particle.best_fitness = particle.fitness
        if particle.fitness < global_best_fitness:
            global_best_fitness = particle.fitness
            global_best_position = particle.position.copy()
            print(f"    New initial best fitness: {global_best_fitness:.2f} seconds")

    initial_eval_end_time = time.time()
    if global_best_position is None: raise ValueError("Could not determine initial global best.")
    fitness_history.append(global_best_fitness) # <<< NEW: Store initial best fitness
    print(f"Initial Global Best Fitness (Est. Time): {global_best_fitness:.2f} seconds (Eval took {initial_eval_end_time - initial_eval_start_time:.2f}s)")
    print("--- Starting Iterations ---")

    # --- Main PSO Loop ---
    start_loop_time = time.time()
    for iteration in range(max_iterations):
        iter_start_time = time.time()
        current_gbest_in_iter = global_best_fitness
        print(f"\nIteration {iteration + 1}/{max_iterations}")

        for i, particle in enumerate(swarm):
            # Update Velocity & Position
            r1 = np.random.rand(num_dimensions); r2 = np.random.rand(num_dimensions)
            cognitive_velocity = cognitive_coeff * r1 * (particle.best_position - particle.position)
            social_velocity = social_coeff * r2 * (global_best_position - particle.position)
            particle.velocity = (inertia_weight * particle.velocity) + cognitive_velocity + social_velocity
            particle.position = particle.position + particle.velocity
            particle.position = np.clip(particle.position, position_min, position_max)

            # Evaluate Fitness
            try:
                 current_fitness = calculate_realtime_fitness(particle.position, G_pso, alternative_paths_data_pso, stops_gdf_pso)
                 if current_fitness == float('inf'): print(f"  Warning: Particle {i+1} infinite fitness.")
                 particle.fitness = current_fitness
            except Exception as fit_e: print(f"Warning: Fitness calc failed particle {i+1}: {fit_e}"); particle.fitness = float('inf')

            # Update Bests
            if particle.fitness < particle.best_fitness:
                particle.best_fitness = particle.fitness
                particle.best_position = particle.position.copy()
                if particle.best_fitness < global_best_fitness:
                    global_best_fitness = particle.best_fitness
                    global_best_position = particle.best_position.copy()

        fitness_history.append(global_best_fitness) # <<< NEW: Store best fitness for this iteration
        iter_end_time = time.time()
        print(f"Iteration {iteration + 1} Complete | Best Fitness: {global_best_fitness:.2f}s | Iter Time: {iter_end_time - iter_start_time:.2f}s{' (* New Swarm Best *)' if global_best_fitness < current_gbest_in_iter else ''}")

    end_loop_time = time.time()
    print("--- PSO Finished ---")
    print(f"Total PSO loop time: {end_loop_time - start_loop_time:.2f} seconds")

    # --- Results ---
    print(f"\nBest solution fitness (minimum estimated time): {global_best_fitness:.2f} seconds")
    if global_best_position is not None:
        print(f"Best solution position (encoded): {np.round(global_best_position, 3)}")
        best_choices = decode_choices(global_best_position)
        print(f"Decoded Best Choices (A=Standard, B=Alternative):")
        print(f" -> Segment 1 (DP1->RP1): Path {best_choices[0]}")
        print(f" -> Segment 2 (DP2->RP2): Path {best_choices[1]}")
        print(f" -> Segment 3 (DP3->RP3): Path {best_choices[2]}")
    else: print("Could not determine the best solution position.")

except NameError as ne: print(f"\nPSO Failed: Required variable missing ({ne}).")
except Exception as pso_e: print(f"\nAn error occurred during PSO execution: {pso_e}"); traceback.print_exc()

# Variables available: global_best_position, global_best_fitness, fitness_history


--- Starting PSO Optimization (Based on Real-Time Estimate) ---
Initializing 5 particles...

PSO Failed: Required variable missing (PSO input data (G_pso, alternatives, stops) missing or invalid. Run Cell 7.).


In [ ]:
# Cell: Helper for Static Length Calculation (from Response #105)
import networkx as nx

memo_std_paths_len_only = {} # Use a different cache name
def get_standard_connecting_segment_length(G, start_node, end_node):
    """Calculates or retrieves cached shortest path length between two nodes."""
    if start_node == end_node: return 0.0
    segment_key = (start_node, end_node)
    if segment_key in memo_std_paths_len_only: return memo_std_paths_len_only[segment_key]

    try:
        # Use nx.shortest_path_length for efficiency if only length needed
        length = nx.shortest_path_length(G, start_node, end_node, weight='length')
        memo_std_paths_len_only[segment_key] = length
        return length
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        print(f"Warning: No standard path found {start_node} -> {end_node}")
        memo_std_paths_len_only[segment_key] = float('inf')
        return float('inf')
    except Exception as e:
        print(f"Warning: Error finding standard path length {start_node} -> {end_node}: {e}")
        memo_std_paths_len_only[segment_key] = float('inf')
        return float('inf')

print("Static length helper function defined.")

Static length helper function defined.


In [ ]:
# Cell 11: PSO Result Visualization and Analysis (Zoomed Length & Time Axes)

import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from shapely.geometry import LineString
import networkx as nx
import numpy as np
import itertools # To generate all 8 strategies
import time
import branca.element # For legend
import gc # For cleanup
import traceback

print("\n--- Analyzing and Visualizing PSO Results ---")

# Check if PSO results and necessary data exist
if ('global_best_position' not in locals() or global_best_position is None or
    'fitness_history' not in locals() or not fitness_history or
    'G_pso' not in locals() or G_pso is None or
    'alternative_paths_data_pso' not in locals() or not alternative_paths_data_pso or
    'stops_gdf_pso' not in locals() or stops_gdf_pso.empty):
    print("Error: PSO results or necessary data not found.")
    print("Please ensure Cell 7 (Setup) and Cell 10 (PSO Loop) ran successfully.")
    raise NameError("PSO results/data missing.")
else:
    # --- 1. Convergence Plot ---
    print("\nGenerating Convergence Plot...")
    # ... (Convergence plot code remains the same) ...
    try:
        plt.figure(figsize=(10, 6))
        valid_fitness_history = [f for f in fitness_history if f != float('inf')]
        if valid_fitness_history:
             min_fit = min(valid_fitness_history); max_fit = max(valid_fitness_history)
             if min_fit > 0 and max_fit / min_fit > 10: # Use log scale if large variation
                  plot_data = np.log10(np.maximum(valid_fitness_history, 1e-6))
                  y_label = 'Log10(Best Fitness - Est. Time in Seconds)'
             else:
                  plot_data = valid_fitness_history
                  y_label = 'Best Fitness - Est. Time in Seconds'
             plt.plot(range(len(plot_data)), plot_data, marker='o', linestyle='-')
             plt.title('PSO Convergence')
             plt.xlabel('Iteration (Incl. Initial Eval)')
             plt.ylabel(y_label)
             plt.grid(True); plt.tight_layout(); plt.show()
        else: print("Could not plot convergence: No valid fitness history found.")
    except Exception as e_plot: print(f"Error generating convergence plot: {e_plot}"); traceback.print_exc()

    # --- 2. Optimal Path Map Visualization ---
    print("\nGenerating Optimal Path Map...")
    # ... (Map code remains the same) ...
    if 'map_viz_results' in locals() and map_viz_results: display(map_viz_results)
    else: print("Map visualization skipped or failed.")

    # --- 3. Comparison of All 8 Strategies (Real-Time Time) ---
    print("\nCalculating estimated time for all 8 possible strategies...")
    # ... (Code for calculating strategy_times remains the same) ...
    all_strategies = list(itertools.product(['A', 'B'], repeat=3))
    strategy_times = {} # Stores { 'AAA': time_sec, ... }
    comparison_start_time = time.time()
    if 'segment_start_node' not in locals() or 'segment_end_node' not in locals():
        stop_nodes_list_comp = ox.nearest_nodes(G_pso, X=stops_gdf_pso.geometry.x.tolist(), Y=stops_gdf_pso.geometry.y.tolist())
        segment_start_node = stop_nodes_list_comp[0]; segment_end_node = stop_nodes_list_comp[-1]
    time_calc_successful = True
    for i, strategy_tuple in enumerate(all_strategies):
        strategy_list = list(strategy_tuple); strategy_str = ''.join(strategy_list)
        dummy_pos = np.array([0.6 if choice == 'B' else 0.1 for choice in strategy_list])
        print(f"  Evaluating time for strategy {strategy_str} ({i+1}/8)...")
        try:
            if 'calculate_realtime_fitness' not in globals(): raise NameError("calculate_realtime_fitness not defined")
            est_time = calculate_realtime_fitness(dummy_pos, G_pso, alternative_paths_data_pso, stops_gdf_pso)
            strategy_times[strategy_str] = est_time
            print(f"    -> Est. Time: {est_time:.2f} seconds")
            if est_time == float('inf'): print(f"    WARNING: Fitness returned infinity for strategy {strategy_str}.")
        except Exception as e_comp: print(f"    -> FATAL ERROR calculating fitness for strategy {strategy_str}: {e_comp}"); traceback.print_exc(); strategy_times[strategy_str] = float('inf'); time_calc_successful = False
    comparison_end_time = time.time()
    print(f"\nComparison Time Evaluation Complete. Success: {time_calc_successful}. Time Taken: {comparison_end_time - comparison_start_time:.2f} seconds")
    print(f"DEBUG: strategy_times dictionary: {strategy_times}")

    # --- Print comparison results table (as before) ---
    # ... (Table printing code remains the same) ...

    # --- 4. Bar Chart Visualization of Strategy Times ---
    print("\nGenerating Strategy Comparison Bar Chart (Estimated Time)...")
    # ... (Bar chart code for strategy_times remains the same) ...

    # --- 5. Calculate Static Length for All 8 Strategies ---
    print("\nCalculating static length for all 8 possible strategies...")
    strategy_lengths = {} # Stores { 'AAA': length_m, ... }
    length_calc_successful = True
    # ... (Code for calculating strategy_lengths remains the same) ...
    try:
        dp_start_nodes = sorted(list(alternative_paths_data_pso.keys()))
        if len(dp_start_nodes) != 3: raise ValueError("Alternatives data doesn't have 3 DPs.")
        dp1_node, dp2_node, dp3_node = dp_start_nodes
        rp1_node = alternative_paths_data_pso[dp1_node]['end_node']; rp2_node = alternative_paths_data_pso[dp2_node]['end_node']; rp3_node = alternative_paths_data_pso[dp3_node]['end_node']
        if 'get_standard_connecting_segment_length' not in globals(): raise NameError("Static length helper function not defined.")
        print("  Calculating standard connecting segment lengths...")
        memo_std_paths_len_only.clear()
        len_s0 = get_standard_connecting_segment_length(G_pso, segment_start_node, dp1_node)
        len_s2 = get_standard_connecting_segment_length(G_pso, rp1_node, dp2_node)
        len_s4 = get_standard_connecting_segment_length(G_pso, rp2_node, dp3_node)
        len_s6 = get_standard_connecting_segment_length(G_pso, rp3_node, segment_end_node)
        print(f"  DEBUG: Standard Lengths: S0={len_s0:.2f}, S2={len_s2:.2f}, S4={len_s4:.2f}, S6={len_s6:.2f}")
        if any(l == float('inf') for l in [len_s0, len_s2, len_s4, len_s6]): raise ValueError("Could not calculate all standard connecting segment lengths.")
        total_standard_len = len_s0 + len_s2 + len_s4 + len_s6
        print("  DEBUG: Checking required length keys in alternative_paths_data_pso...")
        for i, dp_node in enumerate([dp1_node, dp2_node, dp3_node]):
            if 'A_length' not in alternative_paths_data_pso[dp_node] or 'B_length' not in alternative_paths_data_pso[dp_node]: raise KeyError(f"Missing A_length or B_length for DP {i+1} (Node {dp_node})")
        print("  DEBUG: All required length keys found.")
        for strategy_tuple in all_strategies:
            strategy_list = list(strategy_tuple); strategy_str = ''.join(strategy_list)
            total_len = 0.0
            try:
                len_a1 = alternative_paths_data_pso[dp1_node]['A_length']; len_b1 = alternative_paths_data_pso[dp1_node]['B_length']
                len_a2 = alternative_paths_data_pso[dp2_node]['A_length']; len_b2 = alternative_paths_data_pso[dp2_node]['B_length']
                len_a3 = alternative_paths_data_pso[dp3_node]['A_length']; len_b3 = alternative_paths_data_pso[dp3_node]['B_length']
                total_len += total_standard_len
                total_len += len_a1 if strategy_list[0] == 'A' else len_b1
                total_len += len_a2 if strategy_list[1] == 'A' else len_b2
                total_len += len_a3 if strategy_list[2] == 'A' else len_b3
                strategy_lengths[strategy_str] = total_len
            except KeyError as ke: print(f"    -> Error: Missing length data for strategy {strategy_str} (KeyError: {ke})"); strategy_lengths[strategy_str] = float('inf'); length_calc_successful=False
            except Exception as e_len: print(f"    -> Error calculating length for strategy {strategy_str}: {e_len}"); strategy_lengths[strategy_str] = float('inf'); length_calc_successful=False
    except Exception as e_len_setup: print(f"Error during static length calculation setup: {e_len_setup}"); length_calc_successful = False
    print(f"\nStatic Length Calculation Complete. Success: {length_calc_successful}")
    print(f"DEBUG: strategy_lengths dictionary: {strategy_lengths}")

    # --- 6. Grouped Bar Chart (Length vs. Time) with Adjusted Y-Axes ---
    print("\nGenerating Grouped Bar Chart (Length vs. Estimated Time - Adjusted Scales)...")
    if strategy_times and strategy_lengths and time_calc_successful and length_calc_successful:
        valid_strategies = [s for s in strategy_times if strategy_times[s] != float('inf') and strategy_lengths.get(s, float('inf')) != float('inf')]
        if valid_strategies:
            labels = valid_strategies
            static_lengths_m = [strategy_lengths[s] for s in labels]
            est_times_s = [strategy_times[s] for s in labels]

            x = np.arange(len(labels))
            width = 0.35

            fig, ax1 = plt.subplots(figsize=(14, 8))

            # Plot Length bars (Left Y-axis)
            color1 = 'tab:grey'; rects1 = ax1.bar(x - width/2, static_lengths_m, width, label='Static Length (m)', color=color1)
            ax1.set_xlabel('Strategy (Choices for DP1, DP2, DP3)'); ax1.set_ylabel('Static Length (meters)', color=color1)
            ax1.tick_params(axis='y', labelcolor=color1); ax1.set_xticks(x); ax1.set_xticklabels(labels, rotation=45, ha='right'); ax1.grid(axis='y', linestyle='--', alpha=0.7)

            # *** NEW: Adjust Y-axis limits for Length (ax1) ***
            min_len = min(static_lengths_m); max_len = max(static_lengths_m); len_range = max_len - min_len
            len_padding = len_range * 0.1 if len_range > 0 else 50 # Add 50m padding if range is 0
            ax1.set_ylim(max(0, min_len - len_padding), max_len + len_padding) # Ensure lower limit not negative
            print(f"DEBUG: Setting Length Y-axis limits to ({max(0, min_len - len_padding):.1f}, {max_len + len_padding:.1f})")
            # *************************************************

            # Create second y-axis for Estimated Time
            ax2 = ax1.twinx(); color2 = 'tab:blue'; rects2 = ax2.bar(x + width/2, est_times_s, width, label='Est. Real-Time (s)', color=color2)
            ax2.set_ylabel('Estimated Time (seconds)', color=color2); ax2.tick_params(axis='y', labelcolor=color2)

            # Adjust Y-axis limits for Time (ax2)
            min_time = min(est_times_s); max_time = max(est_times_s); time_range = max_time - min_time
            time_padding = time_range * 0.1 if time_range > 0 else 10
            ax2.set_ylim(max(0, min_time - time_padding), max_time + time_padding)
            print(f"DEBUG: Setting Time Y-axis limits to ({max(0, min_time - time_padding):.1f}, {max_time + time_padding:.1f})")

            # Highlight PSO best strategy bars
            pso_best_strategy_str = "".join(decode_choices(global_best_position))
            try: pso_best_index = labels.index(pso_best_strategy_str); rects1[pso_best_index].set_color('dimgray'); rects2[pso_best_index].set_color('royalblue')
            except ValueError: pass

            fig.tight_layout(); plt.title('Static Length vs. Estimated Real-Time Travel Time by Strategy (Zoomed Axes)')
            lines, labels_l = ax1.get_legend_handles_labels(); lines2, labels2 = ax2.get_legend_handles_labels()
            ax2.legend(lines + lines2, labels_l + labels2, loc='upper left')
            plt.show()
        else: print("No valid strategies found with both length and time data to plot.")
    else: print("Could not generate Length vs. Time plot: Missing or invalid data from calculation steps.")

# --- Cleanup ---
# ... (Optional Cleanup code) ...

print("\nCell 11: Results Analysis and Visualization Complete.")


--- Analyzing and Visualizing PSO Results ---
Error: PSO results or necessary data not found.
Please ensure Cell 7 (Setup) and Cell 10 (PSO Loop) ran successfully.


NameError: PSO results/data missing.

In [ ]:
# Cell 12: Inspect Static Segment Lengths

import networkx as nx
import numpy as np
import osmnx as ox # If needed to get start/end nodes again
import traceback

print("\n--- Inspecting Static Lengths of Path Segments ---")

# Check if necessary data is loaded from Cell 7
if ('G_pso' not in locals() or G_pso is None or
    'alternative_paths_data_pso' not in locals() or not alternative_paths_data_pso or
    'stops_gdf_pso' not in locals() or stops_gdf_pso.empty):
    print("Error: Required data (G_pso, alternatives, stops) not found.")
    print("Please ensure Cell 7 (PSO Setup) ran successfully first.")
elif 'get_standard_connecting_segment_length' not in globals():
     print("Error: Helper function 'get_standard_connecting_segment_length' not defined.")
     print("Please ensure the cell defining it (e.g., near Response #121) has been run.")
else:
    try:
        # --- Get Key Nodes ---
        print("Identifying key nodes...")
        dp_start_nodes = sorted(list(alternative_paths_data_pso.keys()))
        if len(dp_start_nodes) != 3: raise ValueError("Alternatives data doesn't have 3 DPs.")
        dp1_node, dp2_node, dp3_node = dp_start_nodes
        rp1_node = alternative_paths_data_pso[dp1_node]['end_node']
        rp2_node = alternative_paths_data_pso[dp2_node]['end_node']
        rp3_node = alternative_paths_data_pso[dp3_node]['end_node']

        # Get start/end nodes (recalculate if not global)
        if 'segment_start_node' not in locals() or 'segment_end_node' not in locals():
             stop_nodes_list_insp = ox.nearest_nodes(G_pso, X=stops_gdf_pso.geometry.x.tolist(), Y=stops_gdf_pso.geometry.y.tolist())
             segment_start_node = stop_nodes_list_insp[0]; segment_end_node = stop_nodes_list_insp[-1]
        print(f"  Start Node: {segment_start_node}, End Node: {segment_end_node}")
        print(f"  DP Nodes: {dp1_node}, {dp2_node}, {dp3_node}")
        print(f"  RP Nodes: {rp1_node}, {rp2_node}, {rp3_node}")

        # --- Calculate and Print Standard Connecting Segment Lengths ---
        print("\nCalculating standard connecting segment lengths...")
        memo_std_paths_len_only.clear() # Clear cache for fresh calculation if needed
        len_s0 = get_standard_connecting_segment_length(G_pso, segment_start_node, dp1_node)
        len_s2 = get_standard_connecting_segment_length(G_pso, rp1_node, dp2_node)
        len_s4 = get_standard_connecting_segment_length(G_pso, rp2_node, dp3_node)
        len_s6 = get_standard_connecting_segment_length(G_pso, rp3_node, segment_end_node)

        print(f"  Length Start -> DP1 (Seg 0): {len_s0:.2f} m")
        print(f"  Length RP1  -> DP2 (Seg 2): {len_s2:.2f} m")
        print(f"  Length RP2  -> DP3 (Seg 4): {len_s4:.2f} m")
        print(f"  Length RP3  -> End  (Seg 6): {len_s6:.2f} m")
        total_standard_len = len_s0 + len_s2 + len_s4 + len_s6
        print(f"  -> Total length of standard connecting segments: {total_standard_len:.2f} m")

        # --- Print Pre-calculated A/B Segment Lengths ---
        print("\nPre-calculated lengths for A/B segments (from Cell 6):")
        len_a1 = alternative_paths_data_pso[dp1_node].get('A_length', float('nan'))
        len_b1 = alternative_paths_data_pso[dp1_node].get('B_length', float('nan'))
        len_a2 = alternative_paths_data_pso[dp2_node].get('A_length', float('nan'))
        len_b2 = alternative_paths_data_pso[dp2_node].get('B_length', float('nan'))
        len_a3 = alternative_paths_data_pso[dp3_node].get('A_length', float('nan'))
        len_b3 = alternative_paths_data_pso[dp3_node].get('B_length', float('nan'))

        print(f"  Segment 1 (DP1 -> RP1): Path A = {len_a1:.2f} m, Path B = {len_b1:.2f} m, Diff = {abs(len_a1 - len_b1):.2f} m")
        print(f"  Segment 3 (DP2 -> RP2): Path A = {len_a2:.2f} m, Path B = {len_b2:.2f} m, Diff = {abs(len_a2 - len_b2):.2f} m")
        print(f"  Segment 5 (DP3 -> RP3): Path A = {len_a3:.2f} m, Path B = {len_b3:.2f} m, Diff = {abs(len_a3 - len_b3):.2f} m")

        # --- Recalculate Total Lengths for Verification ---
        print("\nRecalculated total static lengths for each strategy:")
        valid_lengths_exist = True
        if any(np.isnan(l) or l == float('inf') for l in [len_a1, len_b1, len_a2, len_b2, len_a3, len_b3, len_s0, len_s2, len_s4, len_s6]):
             valid_lengths_exist = False
             print("  Cannot recalculate totals due to missing/infinite segment lengths.")

        if valid_lengths_exist:
             lengths_recalc = {}
             lengths_recalc['AAA'] = total_standard_len + len_a1 + len_a2 + len_a3
             lengths_recalc['AAB'] = total_standard_len + len_a1 + len_a2 + len_b3
             lengths_recalc['ABA'] = total_standard_len + len_a1 + len_b2 + len_a3
             lengths_recalc['ABB'] = total_standard_len + len_a1 + len_b2 + len_b3
             lengths_recalc['BAA'] = total_standard_len + len_b1 + len_a2 + len_a3
             lengths_recalc['BAB'] = total_standard_len + len_b1 + len_a2 + len_b3
             lengths_recalc['BBA'] = total_standard_len + len_b1 + len_b2 + len_a3
             lengths_recalc['BBB'] = total_standard_len + len_b1 + len_b2 + len_b3

             print("Strategy | Recalculated Total Length (m)")
             print("--------|------------------------------")
             for strategy, length in sorted(lengths_recalc.items(), key=lambda item: item[1]):
                 print(f"{strategy:<8} | {length:>28.2f}")

             # Compare with strategy_lengths from Cell 11 if it exists
             if 'strategy_lengths' in locals() and strategy_lengths:
                  print("\nComparing with lengths calculated in Cell 11:")
                  match = True
                  for s in lengths_recalc:
                       if abs(lengths_recalc[s] - strategy_lengths.get(s, -1)) > 0.1: # Allow small float difference
                            print(f"  Mismatch for {s}: Recalculated={lengths_recalc[s]:.2f}, Cell 11={strategy_lengths.get(s):.2f}")
                            match = False
                  if match: print("  Lengths match those calculated in Cell 11.")
             else:
                  print("\n(strategy_lengths dictionary from Cell 11 not found for comparison)")

    except Exception as e_inspect:
        print(f"An error occurred during length inspection: {e_inspect}")
        traceback.print_exc()

print("\nCell 12: Static Length Inspection Complete.")